In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple, Optional
cuda0 = torch.device("cuda")

In [ ]:
def tensor_basics_demo():
    """Demonstrate fundamental tensor operations"""
    # TODO: Create tensors from different sources
    # 1. From Python list
    list_tensor = torch.tensor(
        [
            23,55,23,5,234,735,3,0.2
        ]
    )
    print(list_tensor)
    
    # 2. From NumPy array
    np_array = np.array([[1, 2, 3], [4, 5, 6]])
    numpy_tensor = torch.from_numpy(np_array)
    print(numpy_tensor)
    
    # 3. Random tensor with specific shape
    random_tensor = torch.rand((4,4))
    random_tensor.to(cuda0, dtype=torch.float64)
    print(random_tensor)
    print(random_tensor@random_tensor)
    print(random_tensor.device)
    print(torch.mm(random_tensor,random_tensor))
    # TODO: Perform basic operations
    # Addition, multiplication, matrix operations
    
    add_tensor_L = list_tensor + list_tensor
    print(add_tensor_L)
    
    add_tensor_L.to(cuda0, dtype = torch.float64)
    # TODO: Demonstrate tensor properties
    # Shape, dtype, device
    
    return list_tensor, numpy_tensor, random_tensor

In [ ]:
if __name__ == "__main__":
    tensor_basics_demo()

In [ ]:
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

class MaxFlightTime:
    def __init__(self, velocity=20.0, gravity=9.81):
        self.v = velocity
        self.g = gravity
        # Optimisable parameter: angle
        self.angle = torch.tensor(torch.pi/4, requires_grad=True)  # start at 45°
    
    def flight_time(self):
        return 2 * self.v * torch.sin(self.angle) / self.g
    
    def loss_function(self):
        # We want to maximise time → minimise negative time
        return -self.flight_time()
    
    def optimise(self, steps=200, lr=0.1):
        opt = optim.Adam([self.angle], lr=lr)
        times, losses = [], []
        for _ in range(steps):
            opt.zero_grad()
            loss = self.loss_function()
            loss.backward()
            opt.step()
            times.append(self.flight_time().item())
            losses.append(loss.item())
        return times, losses

# Run optimisation
opt = MaxFlightTime(velocity=20.0)
times, losses = opt.optimise()

# Plot flight time improvement
plt.plot(times)
plt.xlabel("Iteration")
plt.ylabel("Flight time (s)")
plt.title("Maximising projectile flight time")
plt.show()

print("Optimal angle (deg):", opt.angle.item()*180/torch.pi)
print("Maximum flight time (s):", times[-1])


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

class FlightRangeOptimiser:
    def __init__(self, velocity=20.0, gravity=9.81, w_time=0.5, w_range=0.5):
        self.v = velocity
        self.g = gravity
        self.angle = torch.tensor(torch.pi/10, requires_grad=True)  # initial guess
        self.w_time = w_time
        self.w_range = w_range
    
    def flight_time(self):
        return 2 * self.v * torch.sin(self.angle) / self.g
    
    def range(self):
        return self.v**2 * torch.sin(2*self.angle) / self.g
    
    def loss(self):
        # minimise negative weighted sum
        return -(self.w_time*self.flight_time() + self.w_range*self.range())
    
    def optimise(self, steps=300, lr=0.05):
        opt = optim.Adam([self.angle], lr=lr)
        times, ranges = [], []
        for _ in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()
            times.append(self.flight_time().item())
            ranges.append(self.range().item())
        return times, ranges

    def trajectory(self, n_points=200):
        θ = self.angle.item()
        t_flight = 2 * self.v * np.sin(θ) / self.g
        t = np.linspace(0, t_flight, n_points)
        x = self.v * np.cos(θ) * t
        y = self.v * np.sin(θ) * t - 0.5 * self.g * t**2
        return x, y

# --- Run with velocity fixed ---
opt = FlightRangeOptimiser(velocity=20.0, w_time=0.5, w_range=0.5)
times, ranges = opt.optimise()

print("Optimised angle (deg):", opt.angle.item()*180/np.pi)
print("Flight time (s):", times[-1])
print("Range (m):", ranges[-1])

# Plot optimisation history
plt.figure()
plt.plot(times, label="Flight time")
plt.plot(ranges, label="Range")
plt.xlabel("Iteration")
plt.ylabel("Value")
plt.legend()
plt.title("Optimisation progress")
plt.show()

# Plot trajectories every 10 iterations
plt.figure()
for i in range(0, len(times), 10):
    θ = float(opt.angle.item())  # after last step
    # reconstruct trajectory at that sampled angle
    t_f = 2*opt.v*np.sin(θ)/opt.g
    t = np.linspace(0, t_f, 200)
    x = opt.v*np.cos(θ)*t
    y = opt.v*np.sin(θ)*t - 0.5*opt.g*t**2
    plt.plot(x, y, alpha=0.5)

plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("Sampled trajectories during optimisation")
plt.show()

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

class ProjectileTargetOptimiser:
    def __init__(self, target_x, target_y, gravity=9.81):
        self.target_x = target_x
        self.target_y = target_y
        self.g = gravity

        # Optimisable parameters
        self.velocity = torch.tensor(15.0, requires_grad=True)  # initial guess
        self.angle = torch.tensor(torch.pi/4, requires_grad=True)  # initial guess

    def endpoint(self):
        """Compute x position when projectile passes through target_y plane"""
        v, θ, g = self.velocity, self.angle, self.g

        # Solve quadratic: y(t) = v*sinθ*t - 0.5*g*t² = target_y
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y
        disc = b**2 - 4*a*c

        if disc < 0:
            return torch.tensor(float("inf"))  # unreachable
        t1 = (-b + torch.sqrt(disc)) / (2*a)
        t2 = (-b - torch.sqrt(disc)) / (2*a)
        t_hit = torch.max(t1, t2)  # take positive root

        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def loss(self):
        """Squared distance in x at target_y"""
        x_hit = self.endpoint()
        return (x_hit - self.target_x)**2

    def optimise(self, steps=400, lr=0.05):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        history = []
        for _ in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()
            history.append(L.item())
        return history

    def trajectory(self, n_points=200):
        """Full trajectory for plotting"""
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        return x, y


# Example: target at (30, 5)
opt = ProjectileTargetOptimiser(target_x=30.0, target_y=5.0)
losses = opt.optimise()

print("Optimised velocity:", opt.velocity.item())
print("Optimised angle (deg):", opt.angle.item()*180/np.pi)

# Plot convergence
plt.figure()
plt.plot(losses)
plt.xlabel("Iteration")
plt.ylabel("Loss (squared x-distance)")
plt.title("Optimisation towards target")
plt.show()

# Plot trajectory vs target
x, y = opt.trajectory()
plt.figure()
plt.plot(x, y, label="Trajectory")
plt.scatter(opt.target_x, opt.target_y, c="red", marker="x", s=100, label="Target")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.legend()
plt.title("Projectile hitting target")
plt.show()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

class ProjectileTargetOptimiser:
    def __init__(self, target_x, target_y, gravity=9.81):
        self.target_x = target_x
        self.target_y = target_y
        self.g = gravity

        # Optimisable parameters
        self.velocity = torch.tensor(15.0, requires_grad=True)
        self.angle = torch.tensor(torch.pi/4, requires_grad=True)

    def endpoint(self):
        v, θ, g = self.velocity, self.angle, self.g

        # Solve quadratic: y(t) = v*sinθ*t - 0.5*g*t² = target_y
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y
        disc = b**2 - 4*a*c

        if disc < 0:
            return torch.tensor(float("inf"))
        t1 = (-b + torch.sqrt(disc)) / (2*a)
        t2 = (-b - torch.sqrt(disc)) / (2*a)
        t_hit = torch.max(t1, t2)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def loss(self):
        x_hit = self.endpoint()
        return (x_hit - self.target_x)**2

    def trajectory(self, n_points=200):
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        return x, y

    def optimise_and_plot(self, steps=200, lr=0.05, plot_every=10):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        losses = []

        plt.figure()
        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()
            losses.append(L.item())

            # Plot every 'plot_every' steps including first
            if step % plot_every == 0:
                x, y = self.trajectory()
                plt.plot(x, y, alpha=0.6, label=f"Step {step}")

        # Mark target
        plt.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
        plt.xlabel("x (m)")
        plt.ylabel("y (m)")
        plt.title("Projectile optimisation: sampled trajectories")
        plt.show()

        return losses


# Example usage
opt = ProjectileTargetOptimiser(target_x=30.0, target_y=5.0)
losses = opt.optimise_and_plot()

print("Final velocity:", opt.velocity.item())
print("Final angle (deg):", opt.angle.item()*180/np.pi)

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
import os

class ProjectileTargetOptimiser:
    def __init__(self, target_x, target_y, gravity=9.81):
        self.target_x = target_x
        self.target_y = target_y
        self.g = gravity

        # Optimisable parameters
        self.velocity = torch.tensor(15.0, requires_grad=True)
        self.angle = torch.tensor(torch.pi/4, requires_grad=True)

    def endpoint(self):
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y
        disc = b**2 - 4*a*c
        if disc < 0:
            return torch.tensor(float("inf"))
        t1 = (-b + torch.sqrt(disc)) / (2*a)
        t2 = (-b - torch.sqrt(disc)) / (2*a)
        t_hit = torch.max(t1, t2)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def loss(self):
        x_hit = self.endpoint()
        return (x_hit - self.target_x)**2

    def trajectory(self, n_points=200):
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        return x, y

    def optimise_and_gif(self, steps=100, lr=0.05, gif_name="optimisation.gif"):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Print speed + angle
            print(f"Step {step}: v={self.velocity.item():.2f}, angle={self.angle.item()*180/np.pi:.2f} deg")

            # Plot trajectory
            fig, ax = plt.subplots()
            x, y = self.trajectory()
            ax.plot(x, y, label=f"Step {step}")
            ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
            ax.set_xlabel("x (m)")
            ax.set_ylabel("y (m)")
            ax.set_title("Projectile optimisation")
            ax.legend()

            # Save frame to array
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        # Write GIF
        imageio.mimsave(gif_name, frames, fps=5)
        print(f"Saved {gif_name}")


# Example usage
opt = ProjectileTargetOptimiser(target_x=30.0, target_y=5.0)
opt.optimise_and_gif()

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio

class ProjectileTargetOptimiser:
    def __init__(self, target_x, target_y, gravity=9.81):
        self.target_x = target_x
        self.target_y = target_y
        self.g = gravity

        # Optimisable parameters
        self.velocity = torch.tensor(15.0, requires_grad=True)
        self.angle = torch.tensor(torch.pi/4, requires_grad=True)

    def endpoint(self):
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y
        disc = b**2 - 4*a*c
        if disc < 0:
            return torch.tensor(float("inf"))
        t1 = (-b + torch.sqrt(disc)) / (2*a)
        t2 = (-b - torch.sqrt(disc)) / (2*a)
        t_hit = torch.max(t1, t2)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def loss(self):
        x_hit = self.endpoint()
        return (x_hit - self.target_x)**2

    def trajectory(self, n_points=200):
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        return x, y

    def optimise_and_gif(self, steps=500, lr=0.05, gif_name="optimisation.gif", save_every=20):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        all_trajectories = []

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Print speed + angle
            print(f"Step {step}: v={self.velocity.item():.2f}, angle={self.angle.item()*180/np.pi:.2f} deg")

            # Store trajectory
            all_trajectories.append(self.trajectory())

            # Save a frame only every `save_every` steps
            if step % save_every == 0:
                fig, ax = plt.subplots()
                for i, (x, y) in enumerate(all_trajectories):
                    alpha = 0.3 + 0.7*(i/len(all_trajectories))  # later trajectories darker
                    ax.plot(x, y, alpha=alpha)
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Projectile optimisation up to step {step}")
                ax.legend()

                # Convert plot to image array
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        # Write GIF
        imageio.mimsave(gif_name, frames, fps=2)
        print(f"Saved {gif_name}")


# Example usage
opt = ProjectileTargetOptimiser(target_x=30.0, target_y=5.0)
opt.optimise_and_gif()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio

class ProjectileTargetOptimiser:
    def __init__(self, target_x, target_y, gravity=9.81):
        self.target_x = target_x
        self.target_y = target_y
        self.g = gravity

        # Optimisable parameters
        self.velocity = torch.tensor(15.0, requires_grad=True)
        self.angle = torch.tensor(torch.pi/4, requires_grad=True)

    def endpoint(self):
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y
        disc = b**2 - 4*a*c
        if disc < 0:
            return torch.tensor(float("inf"))
        t1 = (-b + torch.sqrt(disc)) / (2*a)
        t2 = (-b - torch.sqrt(disc)) / (2*a)
        t_hit = torch.max(t1, t2)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def loss(self):
        x_hit = self.endpoint()
        return (x_hit - self.target_x)**2

    def trajectory(self, n_points=200):
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        return x, y

    def optimise_and_gif(self, steps=200, lr=0.05, gif_name="optimisation.gif", save_every=20):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Print speed + angle every step
            print(f"Step {step}: v={self.velocity.item():.2f}, angle={self.angle.item()*180/np.pi:.2f} deg")

            # Save trajectory and frame only every `save_every` steps
            if step % save_every == 0:
                sampled_trajectories.append(self.trajectory())

                fig, ax = plt.subplots()
                for i, (x, y) in enumerate(sampled_trajectories):
                    ax.plot(x, y, label=f"Step {i*save_every}")
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Trajectories up to step {step}")
                ax.legend()

                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        # Write GIF
        imageio.mimsave(gif_name, frames, fps=2)
        print(f"Saved {gif_name}")


# Example usage
opt = ProjectileTargetOptimiser(target_x=30.0, target_y=5.0)
opt.optimise_and_gif()

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio

class ProjectileTargetOptimiser:
    def __init__(self, target_x, target_y, gravity=9.81):
        self.target_x = target_x
        self.target_y = target_y
        self.g = gravity

        # Optimisable parameters
        self.velocity = torch.tensor(20.0, requires_grad=True)
        self.angle = torch.tensor(torch.pi/3, requires_grad=True)

    def endpoint(self):
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y
        disc = b**2 - 4*a*c

        if disc < 0:
            return None  # unreachable

        sqrt_disc = torch.sqrt(disc)
        t1 = (-b + sqrt_disc) / (2*a)
        t2 = (-b - sqrt_disc) / (2*a)

        # Time of apex
        t_apex = b / g

        # Smallest positive root < t_apex
        candidates = [t for t in [t1, t2] if t > 0 and t < t_apex]
        if not candidates:
            return None  # cannot hit on ascent

        t_hit = min(candidates)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def loss(self):
        x_hit = self.endpoint()
        if x_hit is None:
            # Differentiable penalty: distance to apex x position
            t_apex = self.velocity * torch.sin(self.angle) / self.g
            x_apex = self.velocity * torch.cos(self.angle) * t_apex
            return (x_apex - self.target_x)**2
        return (x_hit - self.target_x)**2


    def trajectory(self, n_points=500):
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        # apex
        t_apex = v*np.sin(θ)/g
        x_apex = v*np.cos(θ)*t_apex
        y_apex = v*np.sin(θ)*t_apex - 0.5*g*t_apex**2
        return x, y, (x_apex, y_apex)

    def optimise_and_gif(self, steps=500, lr=0.01, gif_name="optimisation.gif", save_every=20):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []

        # Set constant axis limits relative to target
        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            if step % save_every == 0:
                print(f"Step {step}: v={self.velocity.item():.2f}, angle={self.angle.item()*180/np.pi:.2f} deg")
                x, y, apex = self.trajectory()
                sampled_trajectories.append((x, y, apex))

                fig, ax = plt.subplots()
                # Draw older trajectories faint
                for (xi, yi, ai) in sampled_trajectories[:-1]:
                    ax.plot(xi, yi, color="blue", alpha=0.3, linewidth=1)
                    ax.scatter(*ai, color="grey", s=20, alpha=0.5)
                # Highlight latest
                x, y, apex = sampled_trajectories[-1]
                ax.plot(x, y, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(*apex, color="orange", s=40, label="Apex")

                # Target
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")

                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Trajectories up to step {step}")
                ax.legend()

                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=2)
        print(f"Saved {gif_name}")


# Example usage
opt = ProjectileTargetOptimiser(target_x=np.random.randint(0,100), target_y=np.random.randint(0,100))
opt.optimise_and_gif()

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple, Optional

class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y})")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        # Optimisable parameters
        self.velocity = torch.tensor(torch.rand(1).item() * 50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item() * (np.pi/2 - 0.01) + 0.01, requires_grad=True)

    def endpoint(self) -> Optional[torch.Tensor]:
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y

        disc = torch.clamp(b**2 - 4*a*c, min=1e-6)
        sqrt_disc = torch.sqrt(disc)
        t1 = (-b + sqrt_disc) / (2*a)
        t2 = (-b - sqrt_disc) / (2*a)

        t_apex = b / g
        candidates = [t for t in [t1, t2] if t > 0 and t < t_apex]
        if not candidates:
            return None

        t_hit = min(candidates)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def total_flight_time(self) -> torch.Tensor:
        """Total flight time"""
        return 2*self.velocity*torch.sin(self.angle)/self.g

    def loss(self) -> torch.Tensor:
        x_hit = self.endpoint()
        if x_hit is None:
            # penalty if cannot hit target
            t_apex = self.velocity*torch.sin(self.angle)/self.g
            x_apex = self.velocity*torch.cos(self.angle)*t_apex
            target_loss = (x_apex - self.target_x)**2
        else:
            target_loss = (x_hit - self.target_x)**2

        # Multi-objective: subtract flight time (to maximise)
        return self.w_target*target_loss - self.w_time*self.total_flight_time()

    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray, Tuple[float,float]]:
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        t_apex = v*np.sin(θ)/g
        x_apex = v*np.cos(θ)*t_apex
        y_apex = v*np.sin(θ)*t_apex - 0.5*g*t_apex**2
        return x, y, (x_apex, y_apex)

    def optimise_and_gif(
        self,
        steps: int = 1000000,
        lr: float = 0.05,
        gif_name: str = "optimisation_multi.gif",
        save_every: int = 100,
        early_stop_tol: float = 1e-6
    ):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []
        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Bound parameters
            self.velocity.data.clamp_(0.1, 50.0)
            self.angle.data.clamp_(0.01, np.pi/2-0.01)

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

            if step % save_every == 0:
                print(f"Step {step}: v={self.velocity.item():.2f}, angle={self.angle.item()*180/np.pi:.2f} deg")
                x, y, apex = self.trajectory()
                sampled_trajectories.append((x, y, apex))

                fig, ax = plt.subplots()
                n_traj = len(sampled_trajectories)
                cmap = plt.get_cmap("Blues")

                for i, (xi, yi, ai) in enumerate(sampled_trajectories[:-1]):
                    alpha = 0.3 + 0.7*(i/(n_traj-1)) if n_traj > 1 else 0.5
                    ax.plot(xi, yi, color=cmap(alpha), linewidth=1)
                    ax.scatter(*ai, color="grey", s=20, alpha=0.5)

                # Latest trajectory
                x, y, apex = sampled_trajectories[-1]
                ax.plot(x, y, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(*apex, color="orange", s=40, label="Apex")

                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Trajectories up to step {step}")
                if step == 0:
                    ax.legend()
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=2)
        print(f"Saved {gif_name}")


# Example usage
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(target_x=np.random.randint(0,100), target_y=np.random.randint(0,100))
    opt.optimise_and_gif()

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple, Optional

class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y})")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        # Optimisable parameters
        self.velocity = torch.tensor(torch.rand(1).item() * 50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item() * (np.pi/2 - 0.01) + 0.01, requires_grad=True)

    def endpoint(self) -> Optional[torch.Tensor]:
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y

        disc = torch.clamp(b**2 - 4*a*c, min=1e-6)
        sqrt_disc = torch.sqrt(disc)
        t1 = (-b + sqrt_disc) / (2*a)
        t2 = (-b - sqrt_disc) / (2*a)

        t_apex = b / g
        candidates = [t for t in [t1, t2] if t > 0 and t < t_apex]
        if not candidates:
            return None

        t_hit = min(candidates)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def total_flight_time(self) -> torch.Tensor:
        return 2*self.velocity*torch.sin(self.angle)/self.g

    def loss(self) -> torch.Tensor:
        x_hit = self.endpoint()
        if x_hit is None:
            t_apex = self.velocity*torch.sin(self.angle)/self.g
            x_apex = self.velocity*torch.cos(self.angle)*t_apex
            target_loss = (x_apex - self.target_x)**2
        else:
            target_loss = (x_hit - self.target_x)**2
        return self.w_target*target_loss - self.w_time*self.total_flight_time()

    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray, Tuple[float,float]]:
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        t_apex = v*np.sin(θ)/g
        x_apex = v*np.cos(θ)*t_apex
        y_apex = v*np.sin(θ)*t_apex - 0.5*g*t_apex**2
        return x, y, (x_apex, y_apex)

    def optimise_and_gif(
        self,
        steps: int = 1000000,
        lr: float = 0.05,
        gif_name: str = "optimisation_multi.gif",
        save_every: int = 100,
        early_stop_tol: float = 1e-6
    ):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []
        xlim = (0, 1.5*self.target_x)
        ylim = (-1.5*abs(self.target_y), 1.5*abs(self.target_y))
        final_trajectory = None

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Bound parameters
            self.velocity.data.clamp_(0.1, 100.0)
            self.angle.data.clamp_(-np.pi/2 + 0.01, np.pi/2 - 0.01)

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                final_trajectory = self.trajectory()
                break

            if step % save_every == 0:
                v = self.velocity.item()
                θ = self.angle.item()
                v_x = v * np.cos(θ)
                v_y = v * np.sin(θ)
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, "
                      f"v_x={v_x:.2f}, v_y={v_y:.2f}")

                x, y, apex = self.trajectory()
                sampled_trajectories.append((x, y, apex))

                fig, ax = plt.subplots()
                n_traj = len(sampled_trajectories)
                cmap = plt.get_cmap("Blues")

                # Older trajectories
                for i, (xi, yi, ai) in enumerate(sampled_trajectories[:-1]):
                    alpha = 0.3 + 0.7*(i/(n_traj-1)) if n_traj > 1 else 0.5
                    ax.plot(xi, yi, color=cmap(alpha), linewidth=1)
                    ax.scatter(*ai, color="grey", s=20, alpha=0.5)

                # Latest trajectory
                x, y, apex = sampled_trajectories[-1]
                ax.plot(x, y, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(*apex, color="orange", s=40, label="Apex")

                # Target
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Trajectories up to step {step}")
                if step == 0:
                    ax.legend()
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        # Plot final trajectory in thick green on all frames
        if final_trajectory is None:
            final_trajectory = self.trajectory()
        x_final, y_final, _ = final_trajectory

        fig, ax = plt.subplots()
        for (xi, yi, _) in sampled_trajectories:
            ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)
        ax.plot(x_final, y_final, color='green', linewidth=4, label="Final trajectory")
        ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.set_xlabel("x (m)")
        ax.set_ylabel("y (m)")
        ax.set_title("Final Optimised Trajectory")
        ax.legend()
        fig.tight_layout()
        fig.canvas.draw()
        frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        frames.append(frame)
        plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=2)
        print(f"Saved {gif_name}")


# Example usage
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(0,100)
    )
    opt.optimise_and_gif()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple, Optional

class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y})")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        self.velocity = torch.tensor(torch.rand(1).item() * 50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item() * (np.pi/2 - 0.01) + 0.01, requires_grad=True)

    def endpoint(self) -> Optional[torch.Tensor]:
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y

        disc = torch.clamp(b**2 - 4*a*c, min=1e-6)
        sqrt_disc = torch.sqrt(disc)
        t1 = (-b + sqrt_disc) / (2*a)
        t2 = (-b - sqrt_disc) / (2*a)

        t_apex = b / g
        candidates = [t for t in [t1, t2] if t > 0 and t < t_apex]
        if not candidates:
            return None

        t_hit = min(candidates)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def total_flight_time(self) -> torch.Tensor:
        return 2*self.velocity*torch.sin(self.angle)/self.g

    def loss(self) -> torch.Tensor:
        x_hit = self.endpoint()
        if x_hit is None:
            t_apex = self.velocity*torch.sin(self.angle)/self.g
            x_apex = self.velocity*torch.cos(self.angle)*t_apex
            target_loss = (x_apex - self.target_x)**2
        else:
            target_loss = (x_hit - self.target_x)**2
        return self.w_target*target_loss - self.w_time*self.total_flight_time()

    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray, Tuple[float,float]]:
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        t_apex = v*np.sin(θ)/g
        x_apex = v*np.cos(θ)*t_apex
        y_apex = v*np.sin(θ)*t_apex - 0.5*g*t_apex**2
        return x, y, (x_apex, y_apex)

    def optimise_and_gif(
        self,
        steps: int = 1000000,
        lr: float = 0.05,
        gif_name: str = "optimisation_multi_info.gif",
        save_every: int = 100,
        early_stop_tol: float = 1e-9
    ):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []
        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)
        final_trajectory = None

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            self.velocity.data.clamp_(0.1, 50.0)
            self.angle.data.clamp_(0.01, np.pi/2-0.01)

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                final_trajectory = self.trajectory()
                break

            if step % save_every == 0:
                v = self.velocity.item()
                θ = self.angle.item()
                v_x = v * np.cos(θ)
                v_y = v * np.sin(θ)
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, "
                      f"v_x={v_x:.2f}, v_y={v_y:.2f}")

                x, y, apex = self.trajectory()
                sampled_trajectories.append((x, y, apex))

                fig, ax = plt.subplots()
                n_traj = len(sampled_trajectories)
                cmap = plt.get_cmap("Blues")

                for i, (xi, yi, ai) in enumerate(sampled_trajectories[:-1]):
                    alpha = 0.3 + 0.7*(i/(n_traj-1)) if n_traj > 1 else 0.5
                    ax.plot(xi, yi, color=cmap(alpha), linewidth=1)
                    ax.scatter(*ai, color="grey", s=20, alpha=0.5)

                x, y, apex = sampled_trajectories[-1]
                ax.plot(x, y, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(*apex, color="orange", s=40, label="Apex")

                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")

                # Overlay speed, angle, and velocity components
                info_text = f"v={v:.2f} m/s\nangle={θ*180/np.pi:.2f}°\nv_x={v_x:.2f}\nv_y={v_y:.2f}"
                ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                        fontsize=10, verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Trajectories up to step {step}")
                if step == 0:
                    ax.legend()
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        # Animate final trajectory dynamically with info overlay
        if final_trajectory is None:
            final_trajectory = self.trajectory()
        x_final, y_final, _ = final_trajectory
        n_points = len(x_final)
        for i in range(0, n_points, max(1, n_points//100)):
            fig, ax = plt.subplots()
            for (xi, yi, _) in sampled_trajectories:
                ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)

            ax.plot(x_final[:i+1], y_final[:i+1], color='green', linewidth=4)
            ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100)

            info_text = f"v={self.velocity.item():.2f} m/s\nangle={self.angle.item()*180/np.pi:.2f}°\n" \
                        f"v_x={self.velocity.item()*np.cos(self.angle.item()):.2f}\n" \
                        f"v_y={self.velocity.item()*np.sin(self.angle.item()):.2f}"
            ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                    fontsize=10, verticalalignment='top',
                    bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

            ax.set_xlim(xlim)
            ax.set_ylim(ylim)
            ax.set_xlabel("x (m)")
            ax.set_ylabel("y (m)")
            fig.tight_layout()
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=5)
        print(f"Saved {gif_name}")


# Example usage
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(-100,100)
    )
    opt.optimise_and_gif()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple, Optional

class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y})")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        self.velocity = torch.tensor(torch.rand(1).item() * 50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item() * (np.pi/2 - 0.01) + 0.01, requires_grad=True)

    def endpoint(self) -> Optional[torch.Tensor]:
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y

        disc = torch.clamp(b**2 - 4*a*c, min=1e-6)
        sqrt_disc = torch.sqrt(disc)
        t1 = (-b + sqrt_disc) / (2*a)
        t2 = (-b - sqrt_disc) / (2*a)

        t_apex = b / g
        candidates = [t for t in [t1, t2] if t > 0 and t < t_apex]
        if not candidates:
            return None

        t_hit = min(candidates)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def total_flight_time(self) -> torch.Tensor:
        return 2*self.velocity*torch.sin(self.angle)/self.g

    def loss(self) -> torch.Tensor:
        x_hit = self.endpoint()
        if x_hit is None:
            t_apex = self.velocity*torch.sin(self.angle)/self.g
            x_apex = self.velocity*torch.cos(self.angle)*t_apex
            target_loss = (x_apex - self.target_x)**2
        else:
            target_loss = (x_hit - self.target_x)**2
        return self.w_target*target_loss - self.w_time*self.total_flight_time()

    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray, Tuple[float,float]]:
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        t_apex = v*np.sin(θ)/g
        x_apex = v*np.cos(θ)*t_apex
        y_apex = v*np.sin(θ)*t_apex - 0.5*g*t_apex**2
        return x, y, (x_apex, y_apex)

    def optimise_and_gif(
        self,
        steps: int = 1000000,
        lr: float = 0.01,
        gif_name: str = "optimisation_multi_final.gif",
        save_every: int = 20,
        early_stop_tol: float = 1e-9
    ):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []
        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            self.velocity.data.clamp_(0.1, 100.0)
            #self.angle.data.clamp_(0.01, np.pi/2-0.01)

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

            if step % save_every == 0:
                v = self.velocity.item()
                θ = self.angle.item()
                v_x = v * np.cos(θ)
                v_y = v * np.sin(θ)
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, "
                      f"v_x={v_x:.2f}, v_y={v_y:.2f}")

                x, y, apex = self.trajectory()
                sampled_trajectories.append((x, y, apex))

                fig, ax = plt.subplots()
                n_traj = len(sampled_trajectories)
                cmap = plt.get_cmap("Blues")

                for i, (xi, yi, ai) in enumerate(sampled_trajectories[:-1]):
                    alpha = 0.3 + 0.7*(i/(n_traj-1)) if n_traj > 1 else 0.5
                    ax.plot(xi, yi, color=cmap(alpha), linewidth=1)
                    ax.scatter(*ai, color="grey", s=20, alpha=0.5)

                x, y, apex = sampled_trajectories[-1]
                ax.plot(x, y, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(*apex, color="orange", s=40, label="Apex")
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")

                info_text = f"v={v:.2f} m/s\nangle={θ*180/np.pi:.2f}°\nv_x={v_x:.2f}\nv_y={v_y:.2f}"
                ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                        fontsize=10, verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        # Ensure final trajectory is drawn in thick green
        x_final, y_final, _ = self.trajectory()
        n_points = len(x_final)
        step_size = max(1, n_points // 100)  # max 100 frames for speed
        for i in range(0, n_points, step_size):
            fig, ax = plt.subplots()
            for (xi, yi, _) in sampled_trajectories:
                ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)
            ax.plot(x_final[:i+1], y_final[:i+1], color='green', linewidth=4)
            ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100)

            v = self.velocity.item()
            θ = self.angle.item()
            v_x = v * np.cos(θ)
            v_y = v * np.sin(θ)
            info_text = f"v={v:.2f} m/s\nangle={θ*180/np.pi:.2f}°\nv_x={v_x:.2f}\nv_y={v_y:.2f}"
            ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                    fontsize=10, verticalalignment='top',
                    bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

            ax.set_xlim(xlim)
            ax.set_ylim(ylim)
            ax.set_xlabel("x (m)")
            ax.set_ylabel("y (m)")
            ax.set_title("Final trajectory")
            fig.tight_layout()
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=5)
        print(f"Saved {gif_name}")


# Example usage
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(-100,100)
    )
    opt.optimise_and_gif()


In [ ]:
class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y})")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        self.velocity = torch.tensor(torch.rand(1).item() * 50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item() * (np.pi/2 - 0.01) + 0.01, requires_grad=True)

        self.history_steps = []
        self.history_v = []
        self.history_angle = []
        self.history_apex = []

    def endpoint(self) -> Optional[torch.Tensor]:
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y

        disc = torch.clamp(b**2 - 4*a*c, min=1e-6)
        sqrt_disc = torch.sqrt(disc)
        t1 = (-b + sqrt_disc) / (2*a)
        t2 = (-b - sqrt_disc) / (2*a)

        t_apex = b / g
        candidates = [t for t in [t1, t2] if t > 0 and t < t_apex]
        if not candidates:
            return None

        t_hit = min(candidates)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def total_flight_time(self) -> torch.Tensor:
        return 2*self.velocity*torch.sin(self.angle)/self.g

    def loss(self) -> torch.Tensor:
        x_hit = self.endpoint()
        if x_hit is None:
            t_apex = self.velocity*torch.sin(self.angle)/self.g
            x_apex = self.velocity*torch.cos(self.angle)*t_apex
            target_loss = (x_apex - self.target_x)**2
        else:
            target_loss = (x_hit - self.target_x)**2
        return self.w_target*target_loss - self.w_time*self.total_flight_time()

    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray, Tuple[float,float]]:
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        t_apex = v*np.sin(θ)/g
        x_apex = v*np.cos(θ)*t_apex
        y_apex = v*np.sin(θ)*t_apex - 0.5*g*t_apex**2
        return x, y, (x_apex, y_apex)

    def optimise_and_gif(
        self,
        steps: int = 1000000,
        lr: float = 0.01,
        gif_name: str = "optimisation_multi_final.gif",
        save_every: int = 20,
        early_stop_tol: float = 1e-9
    ):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []
        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            self.velocity.data.clamp_(0.1, 100.0)
            self.angle.data.clamp_(-np.pi/2 + 0.01, np.pi/2-0.01)

            v = self.velocity.item()
            θ = self.angle.item()
            v_x = v * np.cos(θ)
            v_y = v * np.sin(θ)
            x, y, apex = self.trajectory()

            # Store step history
            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_angle.append(θ*180/np.pi)
            self.history_apex.append(apex)

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, "
                      f"v_x={v_x:.2f}, v_y={v_y:.2f}")
                sampled_trajectories.append((x, y, apex))

                fig, ax = plt.subplots()
                n_traj = len(sampled_trajectories)
                cmap = plt.get_cmap("Blues")

                for i, (xi, yi, ai) in enumerate(sampled_trajectories[:-1]):
                    alpha = 0.3 + 0.7*(i/(n_traj-1)) if n_traj > 1 else 0.5
                    ax.plot(xi, yi, color=cmap(alpha), linewidth=1)
                    ax.scatter(*ai, color="grey", s=20, alpha=0.5)

                x, y, apex = sampled_trajectories[-1]
                ax.plot(x, y, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(*apex, color="orange", s=40, label="Apex")
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")

                info_text = f"v={v:.2f} m/s\nangle={θ*180/np.pi:.2f}°\nv_x={v_x:.2f}\nv_y={v_y:.2f}"
                ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                        fontsize=10, verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Trajectories up to step {step}")
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        # Save GIF of final trajectory
        x_final, y_final, _ = self.trajectory()
        n_points = len(x_final)
        step_size = max(1, n_points // 100)
        for i in range(0, n_points, step_size):
            fig, ax = plt.subplots()
            for (xi, yi, _) in sampled_trajectories:
                ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)
            ax.plot(x_final[:i+1], y_final[:i+1], color='green', linewidth=4)
            ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100)
            ax.set_xlim(0, 1.5*self.target_x)
            ax.set_ylim(0, 1.5*self.target_y)
            fig.tight_layout()
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=5)
        print(f"Saved {gif_name}")

        # --- New plots in terminal ---

        # 1. Speed & Angle vs iteration
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(self.history_steps, self.history_v, 'b-', label='Speed (m/s)')
        ax2.plot(self.history_steps, self.history_angle, 'r-', label='Angle (deg)')

        ax1.set_xlabel('Iteration Step')
        ax1.set_ylabel('Speed (m/s)', color='b')
        ax2.set_ylabel('Angle (deg)', color='r')
        ax1.set_title("Speed & Angle vs Step")
        ax1.grid(True)
        fig.tight_layout()
        plt.show()

        # 2. Apex points trajectory
        fig, ax = plt.subplots()
        apex_x = [ap[0] for ap in self.history_apex]
        apex_y = [ap[1] for ap in self.history_apex]
        ax.plot(apex_x, apex_y, 'o-', color='purple', label='Apex')
        ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100, label='Target')
        ax.set_xlabel('x (m)')
        ax.set_ylabel('y (m)')
        ax.set_title('Apex Points Trajectory')
        ax.grid(True)
        ax.legend()
        plt.tight_layout()
        plt.show()


# Example usage
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(0,100)
    )
    opt.optimise_and_gif()

In [ ]:
class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05, angle_penalty_strength: float = 10.0):
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y})")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time
        self.angle_penalty_strength = angle_penalty_strength

        self.velocity = torch.tensor(torch.rand(1).item() * 50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item() * (np.pi/2 - 0.01) + 0.01, requires_grad=True)

        self.history_steps = []
        self.history_v = []
        self.history_angle = []
        self.history_nearest_point = []

    def endpoint(self) -> Optional[torch.Tensor]:
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y
        disc = torch.clamp(b**2 - 4*a*c, min=1e-6)
        sqrt_disc = torch.sqrt(disc)
        t1 = (-b + sqrt_disc) / (2*a)
        t2 = (-b - sqrt_disc) / (2*a)
        t_apex = b / g
        candidates = [t for t in [t1, t2] if t > 0 and t < t_apex]
        if not candidates:
            return None
        t_hit = min(candidates)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def total_flight_time(self) -> torch.Tensor:
        return 2*self.velocity*torch.sin(self.angle)/self.g

    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray]:
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        return x, y

    def loss(self) -> torch.Tensor:
        x_hit = self.endpoint()
        if x_hit is None:
            t_apex = self.velocity*torch.sin(self.angle)/self.g
            x_apex = self.velocity*torch.cos(self.angle)*t_apex
            target_loss = (x_apex - self.target_x)**2
        else:
            target_loss = (x_hit - self.target_x)**2

        # Uniform penalty for angles between 40° and 50°
        θ_deg = self.angle * 180 / np.pi
        angle_penalty = torch.tensor(0.0)
        if 40 <= θ_deg <= 50:
            angle_penalty = self.angle_penalty_strength

        return self.w_target*target_loss - self.w_time*self.total_flight_time() + angle_penalty

    def optimise_and_gif(
        self,
        steps: int = 100000,
        lr: float = 0.05,
        gif_name: str = "optimisation_multi_final.gif",
        save_every: int = 20,
        early_stop_tol: float = 1e-7
    ):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []
        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            self.velocity.data.clamp_(0.1, 100.0)
            self.angle.data.clamp_(-np.pi/2 + 0.01, np.pi/2-0.01)

            v = self.velocity.item()
            θ = self.angle.item()
            x, y = self.trajectory()

            # Store history
            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_angle.append(θ*180/np.pi)

            # Nearest point to target
            distances = np.sqrt((x - self.target_x)**2 + (y - self.target_y)**2)
            nearest_idx = np.argmin(distances)
            self.history_nearest_point.append((x[nearest_idx], y[nearest_idx]))

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, "
                      f"v_x={v*np.cos(θ):.2f}, v_y={v*np.sin(θ):.2f}")
                sampled_trajectories.append((x, y))

                # Plot GIF frame
                fig, ax = plt.subplots()
                for xi, yi in sampled_trajectories[:-1]:
                    ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)
                ax.plot(x, y, color='black', linewidth=2, label=f"Step {step}")

                nearest_point = self.history_nearest_point[-1]
                ax.scatter(*nearest_point, color='orange', s=50, label='Nearest Point')
                ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100, label='Target')

                info_text = f"v={v:.2f} m/s\nangle={θ*180/np.pi:.2f}°\nv_x={v*np.cos(θ):.2f}\nv_y={v*np.sin(θ):.2f}"
                ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                        fontsize=10, verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        # Final trajectory frame (green)
        x_final, y_final = self.trajectory()
        n_points = len(x_final)
        step_size = max(1, n_points // 100)
        for i in range(0, n_points, step_size):
            fig, ax = plt.subplots()
            for xi, yi in sampled_trajectories:
                ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)
            ax.plot(x_final[:i+1], y_final[:i+1], color='green', linewidth=4)
            ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100)
            distances = np.sqrt((x_final[:i+1] - self.target_x)**2 + (y_final[:i+1] - self.target_y)**2)
            nearest_idx = np.argmin(distances)
            ax.scatter(x_final[nearest_idx], y_final[nearest_idx], color='orange', s=50, label='Nearest Point')
            ax.set_xlim(xlim)
            ax.set_ylim(ylim)
            fig.tight_layout()
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=5)
        print(f"Saved {gif_name}")

        # --- Terminal plots ---

        # 1. Speed & Angle vs iteration
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(self.history_steps, self.history_v, 'b-', label='Speed (m/s)')
        ax2.plot(self.history_steps, self.history_angle, 'r-', label='Angle (deg)')
        ax1.set_xlabel('Iteration Step')
        ax1.set_ylabel('Speed (m/s)', color='b')
        ax2.set_ylabel('Angle (deg)', color='r')
        ax1.set_title("Speed & Angle vs Step")
        ax1.grid(True)
        plt.show()

        # 2. Nearest point to target trajectory
        fig, ax = plt.subplots()
        nearest_x = [pt[0] for pt in self.history_nearest_point]
        nearest_y = [pt[1] for pt in self.history_nearest_point]
        ax.plot(nearest_x, nearest_y, 'o-', color='purple', label='Nearest point to target')
        ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100, label='Target')
        ax.set_xlabel('x (m)')
        ax.set_ylabel('y (m)')
        ax.set_title('Trajectory Point Nearest Target Over Iterations')
        ax.grid(True)
        ax.legend()
        plt.show()

# Example usage
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(0,100)
    )
    opt.optimise_and_gif()

In [ ]:
class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05,
                 angle_penalty_strength: float = 100.0):
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y})")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time
        self.angle_penalty_strength = angle_penalty_strength
        self.velocity = torch.tensor(torch.rand(1).item() * 50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item() * (np.pi/2 - 0.01) + 0.01, requires_grad=True)

        self.history_steps = []
        self.history_v = []
        self.history_angle = []
        self.history_apex = []

    def endpoint(self) -> Optional[torch.Tensor]:
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y

        disc = torch.clamp(b**2 - 4*a*c, min=1e-6)
        sqrt_disc = torch.sqrt(disc)
        t1 = (-b + sqrt_disc) / (2*a)
        t2 = (-b - sqrt_disc) / (2*a)

        t_apex = b / g
        candidates = [t for t in [t1, t2] if t > 0 and t < t_apex]
        if not candidates:
            return None

        t_hit = min(candidates)
        x_hit = v * torch.cos(θ) * t_hit
        return x_hit

    def total_flight_time(self) -> torch.Tensor:
        return 2*self.velocity*torch.sin(self.angle)/self.g

    def loss(self) -> torch.Tensor:
        l = 0
        x_hit = self.endpoint()
        if x_hit is None:
            t_apex = self.velocity*torch.sin(self.angle)/self.g
            x_apex = self.velocity*torch.cos(self.angle)*t_apex
            target_loss = (x_apex - self.target_x)**2
        else:
            target_loss = (x_hit - self.target_x)**2

        # Uniform penalty for angles between 40° and 50°
        θ_deg = self.angle * 180 / np.pi
        angle_penalty = torch.tensor(0.0)
        if 44.5 <= θ_deg <= 45.5:
            angle_penalty = self.angle_penalty_strength
        else:
            angle_penalty = 0.0

        l+=angle_penalty
        l+=self.w_target*target_loss - self.w_time*self.total_flight_time()
        return l


    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray, Tuple[float,float]]:
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        t_apex = v*np.sin(θ)/g
        x_apex = v*np.cos(θ)*t_apex
        y_apex = v*np.sin(θ)*t_apex - 0.5*g*t_apex**2
        return x, y, (x_apex, y_apex)

    def optimise_and_gif(
        self,
        steps: int = 100000,
        lr: float = 0.1,
        gif_name: str = "optimisation_multi_final.gif",
        save_every: int = 100,
        early_stop_tol: float = 1e-8
    ):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []
        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            self.velocity.data.clamp_(0.1, 100.0)
            self.angle.data.clamp_(-np.pi/2 + 0.01, np.pi/2-0.01)

            v = self.velocity.item()
            θ = self.angle.item()
            v_x = v * np.cos(θ)
            v_y = v * np.sin(θ)
            x, y, apex = self.trajectory()

            # Store step history
            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_angle.append(θ*180/np.pi)
            self.history_apex.append(apex)

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, "
                      f"v_x={v_x:.2f}, v_y={v_y:.2f}")
                print(f"Loss: {L.item():.4f}")
                sampled_trajectories.append((x, y, apex))

                fig, ax = plt.subplots()
                n_traj = len(sampled_trajectories)
                cmap = plt.get_cmap("Blues")

                for i, (xi, yi, ai) in enumerate(sampled_trajectories[:-1]):
                    alpha = 0.3 + 0.7*(i/(n_traj-1)) if n_traj > 1 else 0.5
                    ax.plot(xi, yi, color=cmap(alpha), linewidth=1)
                    ax.scatter(*ai, color="grey", s=20, alpha=0.5)

                x, y, apex = sampled_trajectories[-1]
                ax.plot(x, y, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(*apex, color="orange", s=40, label="Apex")
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")

                info_text = f"v={v:.2f} m/s\nangle={θ*180/np.pi:.2f}°\nv_x={v_x:.2f}\nv_y={v_y:.2f}"
                ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                        fontsize=10, verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Trajectories up to step {step}")
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        # Save GIF of final trajectory
        x_final, y_final, _ = self.trajectory()
        n_points = len(x_final)
        step_size = max(1, n_points // 100)
        for i in range(0, n_points, step_size):
            fig, ax = plt.subplots()
            for (xi, yi, _) in sampled_trajectories:
                ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)
            ax.plot(x_final[:i+1], y_final[:i+1], color='green', linewidth=4)
            ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100)
            ax.set_xlim(0, 1.5*self.target_x)
            ax.set_ylim(0, 1.5*self.target_y)
            fig.tight_layout()
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=5)
        print(f"Saved {gif_name}")

        # --- New plots in terminal ---

        # 1. Speed & Angle vs iteration
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(self.history_steps, self.history_v, 'b-', label='Speed (m/s)')
        ax2.plot(self.history_steps, self.history_angle, 'r-', label='Angle (deg)')

        ax1.set_xlabel('Iteration Step')
        ax1.set_ylabel('Speed (m/s)', color='b')
        ax2.set_ylabel('Angle (deg)', color='r')
        ax1.set_title("Speed & Angle vs Step")
        ax1.grid(True)
        fig.tight_layout()
        plt.show()

        # 2. Apex points trajectory
        fig, ax = plt.subplots()
        apex_x = [ap[0] for ap in self.history_apex]
        apex_y = [ap[1] for ap in self.history_apex]
        ax.plot(apex_x, apex_y, 'o-', color='purple', label='Apex')
        ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100, label='Target')
        ax.set_xlabel('x (m)')
        ax.set_ylabel('y (m)')
        ax.set_title('Apex Points Trajectory')
        ax.grid(True)
        ax.legend()
        plt.tight_layout()
        plt.show()


# Example usage
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(0,100)
    )
    opt.optimise_and_gif()

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple, Optional

class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05, w_angle_penalty: float = 50.0):
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y})")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time
        self.w_angle_penalty = w_angle_penalty

        self.velocity = torch.tensor(torch.rand(1).item() * 50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item() * (np.pi/2 - 0.01) + 0.01, requires_grad=True)

        # history
        self.history_steps = []
        self.history_v = []
        self.history_angle = []
        self.history_nearest_point = []

    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray]:
        """Compute full projectile trajectory."""
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2 * v * np.sin(θ) / g
        t = np.linspace(0, t_flight, n_points)
        x = v * np.cos(θ) * t
        y = v * np.sin(θ) * t - 0.5 * g * t**2
        return x, y

    def nearest_point_to_target(self, x: np.ndarray, y: np.ndarray) -> Tuple[float, float]:
        """Find point on trajectory closest to target."""
        distances = np.sqrt((x - self.target_x)**2 + (y - self.target_y)**2)
        idx = np.argmin(distances)
        return x[idx], y[idx]

    def endpoint(self) -> torch.Tensor:
        """Compute x at target_y plane (simplified quadratic solution)."""
        v, θ, g = self.velocity, self.angle, self.g
        a = -0.5 * g
        b = v * torch.sin(θ)
        c = -self.target_y
        disc = b**2 - 4*a*c
        if disc < 0:
            return torch.tensor(float("inf"))
        t1 = (-b + torch.sqrt(disc)) / (2*a)
        t2 = (-b - torch.sqrt(disc)) / (2*a)
        t_hit = torch.max(t1, t2)
        return v * torch.cos(θ) * t_hit

    def total_flight_time(self) -> torch.Tensor:
        return 2*self.velocity*torch.sin(self.angle)/self.g

    def loss(self) -> torch.Tensor:
        x_hit = self.endpoint()
        target_loss = (x_hit - self.target_x)**2

        # uniform penalty for angles between 40 and 50 degrees
        angle_deg = self.angle * 180/np.pi
        angle_penalty = torch.where((angle_deg >= 40) & (angle_deg <= 50),
                                    torch.tensor(self.w_angle_penalty, dtype=torch.float32),
                                    torch.tensor(0.0, dtype=torch.float32))
        return self.w_target*target_loss - self.w_time*self.total_flight_time() + angle_penalty

    def optimise_and_gif(self, steps: int = 100000, lr: float = 0.01,
                         gif_name: str = "optimisation_multi_final.gif",
                         save_every: int = 20, early_stop_tol: float = 1e-9):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []

        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            self.velocity.data.clamp_(0.1, 100.0)
            self.angle.data.clamp_(-np.pi/2 + 0.01, np.pi/2 - 0.01)

            v = self.velocity.item()
            θ = self.angle.item()
            v_x = v * np.cos(θ)
            v_y = v * np.sin(θ)
            x, y = self.trajectory()
            nearest_pt = self.nearest_point_to_target(x, y)

            # store history
            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_angle.append(θ*180/np.pi)
            self.history_nearest_point.append(nearest_pt)

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, "
                      f"v_x={v_x:.2f}, v_y={v_y:.2f}")
                sampled_trajectories.append((x, y))

                # plot current trajectories
                fig, ax = plt.subplots()
                n_traj = len(sampled_trajectories)
                cmap = plt.get_cmap("Blues")
                for i, (xi, yi) in enumerate(sampled_trajectories[:-1]):
                    alpha = 0.3 + 0.7*(i/(n_traj-1)) if n_traj > 1 else 0.5
                    ax.plot(xi, yi, color=cmap(alpha), linewidth=1)
                ax.plot(x, y, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100)
                info_text = f"v={v:.2f} m/s\nangle={θ*180/np.pi:.2f}°\nv_x={v_x:.2f}\nv_y={v_y:.2f}"
                ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                        fontsize=10, verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

        # final trajectory green line
        x_final, y_final = self.trajectory()
        n_points = len(x_final)
        step_size = max(1, n_points//100)
        for i in range(0, n_points, step_size):
            fig, ax = plt.subplots()
            for (xi, yi) in sampled_trajectories:
                ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)
            ax.plot(x_final[:i+1], y_final[:i+1], color='green', linewidth=4)
            ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100)
            ax.set_xlim(xlim)
            ax.set_ylim(ylim)
            fig.tight_layout()
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=5)
        print(f"Saved {gif_name}")

        # --- Terminal plots ---
        # Speed & angle
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(self.history_steps, self.history_v, 'b-', label='Speed (m/s)')
        ax2.plot(self.history_steps, self.history_angle, 'r-', label='Angle (deg)')
        ax1.set_xlabel('Iteration Step')
        ax1.set_ylabel('Speed (m/s)', color='b')
        ax2.set_ylabel('Angle (deg)', color='r')
        fig.tight_layout()
        plt.show()

        # Nearest point to target
        fig, ax = plt.subplots()
        nearest_x = [pt[0] for pt in self.history_nearest_point]
        nearest_y = [pt[1] for pt in self.history_nearest_point]
        ax.plot(nearest_x, nearest_y, 'o-', color='purple', label='Nearest point to target')
        ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100, label='Target')
        ax.set_xlabel('x (m)')
        ax.set_ylabel('y (m)')
        ax.legend()
        plt.tight_layout()
        plt.show()


# Example usage
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(0,100)
    )
    opt.optimise_and_gif()

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple, Optional

class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y})")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        # Initial guesses
        self.velocity = torch.tensor(torch.rand(1).item()*50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item()*np.pi - (np.pi/2) + 0.01, requires_grad=True)

        # History tracking
        self.history_steps = []
        self.history_v = []
        self.history_angle = []
        self.history_apex = []
        self.history_nearest_point = []
        self.history_min_dist = []
        self.history_max_dist = []

    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray, Tuple[float,float]]:
        """Return full trajectory and apex"""
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        t_apex = v*np.sin(θ)/g
        x_apex = v*np.cos(θ)*t_apex
        y_apex = v*np.sin(θ)*t_apex - 0.5*g*t_apex**2
        return x, y, (x_apex, y_apex)

    def nearest_point_to_target(self, x: np.ndarray, y: np.ndarray) -> Tuple[float,float]:
        """Return point on trajectory nearest to target"""
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2
        idx = np.argmin(d2)
        return x[idx], y[idx]

    def max_distance_to_target(self, x: np.ndarray, y: np.ndarray) -> float:
        return float(np.max(np.sqrt((x - self.target_x)**2 + (y - self.target_y)**2)))

    def loss(self) -> torch.Tensor:
        v, θ = self.velocity, self.angle

        # Full trajectory for max distance
        n_points = 501
        t_flight = 2*v*torch.sin(θ)/self.g
        t = torch.linspace(0, t_flight, n_points)
        x = v*torch.cos(θ)*t
        y = v*torch.sin(θ)*t - 0.5*self.g*t**2

        # Distance to target
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2
        min_dist = torch.min(torch.sqrt(d2))
        max_dist = torch.max(torch.sqrt(d2))
        self.current_max_dist = max_dist.item()
        self.current_min_dist = min_dist.item()

        # Target loss = squared distance at closest point
        target_loss = 100*min_dist**2

        # Uniform penalty for angles between 40–50 degrees
        angle_deg = θ*180/np.pi
        penalty = 0.0
        if 44.5 <= angle_deg <= 45.5:
            penalty = 10  # uniform penalty

        # Multi-objective: target + flight time + angle penalty
        total_loss = self.w_target*target_loss - self.w_time*(2*v*torch.sin(θ)/self.g) + penalty
        return total_loss

    def optimise_and_gif(
        self,
        steps: int = 5000,
        lr: float = 0.025,
        gif_name: str = "optimisation_multi_final.gif",
        save_every: int = 25,
        early_stop_tol: float = 1e-9
    ):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []
        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Clamp values
            self.velocity.data.clamp_(0.1, 100.0)
            self.angle.data.clamp_(-np.pi/2 + 0.01, np.pi/2 - 0.01)

            v = self.velocity.item()
            θ = self.angle.item()
            v_x = v * np.cos(θ)
            v_y = v * np.sin(θ)
            x, y, apex = self.trajectory()

            nearest_point = self.nearest_point_to_target(x, y)

            # Store history
            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_angle.append(θ*180/np.pi)
            self.history_apex.append(apex)
            self.history_nearest_point.append(nearest_point)
            self.history_min_dist.append(self.current_min_dist)
            self.history_max_dist.append(self.current_max_dist)

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, "
                      f"v_x={v_x:.2f}, v_y={v_y:.2f}, min_dist={self.history_min_dist[-1]:.2f}")
                sampled_trajectories.append((x, y, apex))

                # Plot snapshot for GIF
                fig, ax = plt.subplots()
                n_traj = len(sampled_trajectories)
                cmap = plt.get_cmap("Blues")
                for i, (xi, yi, ai) in enumerate(sampled_trajectories[:-1]):
                    alpha = 0.3 + 0.7*(i/(n_traj-1)) if n_traj > 1 else 0.5
                    ax.plot(xi, yi, color=cmap(alpha), linewidth=1)
                    ax.scatter(*ai, color="grey", s=20, alpha=0.5)
                # Latest trajectory
                xi, yi, ai = sampled_trajectories[-1]
                ax.plot(xi, yi, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(*ai, color="orange", s=40, label="Apex")
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
                info_text = f"v={v:.2f} m/s\nangle={θ*180/np.pi:.2f}°\nv_x={v_x:.2f}\nv_y={v_y:.2f}\nmin_dist={self.history_min_dist[-1]:.2f}"
                ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                        fontsize=10, verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Trajectories up to step {step}")
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

        # Animate final trajectory
        x_final, y_final, _ = self.trajectory()
        n_points = len(x_final)
        step_size = max(1, n_points//100)
        for i in range(0, n_points, step_size):
            fig, ax = plt.subplots()
            for (xi, yi, _) in sampled_trajectories:
                ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)
            ax.plot(x_final[:i+1], y_final[:i+1], color='green', linewidth=4)
            ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100)
            info_text = f"v={self.velocity.item():.2f} m/s\nangle={self.angle.item()*180/np.pi:.2f}°\n" \
                        f"v_x={self.velocity.item()*np.cos(self.angle.item()):.2f}\n" \
                        f"v_y={self.velocity.item()*np.sin(self.angle.item()):.2f}"
            ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                    fontsize=10, verticalalignment='top',
                    bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
            ax.set_xlim(xlim)
            ax.set_ylim(ylim)
            fig.tight_layout()
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=5)
        print(f"Saved {gif_name}")

        # --- Terminal plots ---
        # 1. Speed & Angle vs iteration
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(self.history_steps, self.history_v, 'b-', label='Speed (m/s)')
        ax2.plot(self.history_steps, self.history_angle, 'r-', label='Angle (deg)')
        ax1.set_xlabel('Iteration Step')
        ax1.set_ylabel('Speed (m/s)', color='b')
        ax2.set_ylabel('Angle (deg)', color='r')
        fig.tight_layout()
        plt.show()

        # 2. Nearest point to target
        fig, ax = plt.subplots()
        nearest_points_x = [p[0] for p in self.history_nearest_point]
        nearest_points_y = [p[1] for p in self.history_nearest_point]
        ax.plot(nearest_points_x, nearest_points_y, 'o-', color='purple', label='Nearest Point')
        ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100, label='Target')
        ax.set_xlabel('x (m)')
        ax.set_ylabel('y (m)')
        ax.legend()
        plt.tight_layout()
        plt.show()

        # 3. Maximum distance vs iteration
        fig, ax = plt.subplots()
        ax.plot(self.history_steps, self.history_max_dist, 'm-', label='Max Distance to Target')
        ax.set_xlabel('Iteration Step')
        ax.set_ylabel('Distance (m)')
        ax.legend()
        plt.tight_layout()
        plt.show()


# --- Example usage ---
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(0,100)
    )
    opt.optimise_and_gif()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple, Optional

class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y}) | Using device: {self.device}")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        # Initial guesses
        self.velocity = torch.tensor(torch.rand(1).item()*50, requires_grad=True)
        self.angle = torch.tensor(torch.rand(1).item()*np.pi - (np.pi/2) + 0.01, requires_grad=True)

        # History tracking
        self.history_steps = []
        self.history_v = []
        self.history_angle = []
        self.history_apex = []
        self.history_nearest_point = []
        self.history_min_dist = []
        self.history_max_dist = []

    def trajectory(self, n_points: int = 501) -> Tuple[np.ndarray, np.ndarray, Tuple[float,float]]:
        """Return full trajectory and apex"""
        v, θ, g = self.velocity.item(), self.angle.item(), self.g
        t_flight = 2*v*np.sin(θ)/g
        t = np.linspace(0, t_flight, n_points)
        x = v*np.cos(θ)*t
        y = v*np.sin(θ)*t - 0.5*g*t**2
        t_apex = v*np.sin(θ)/g
        x_apex = v*np.cos(θ)*t_apex
        y_apex = v*np.sin(θ)*t_apex - 0.5*g*t_apex**2
        return x, y, (x_apex, y_apex)

    def nearest_point_to_target(self, x: np.ndarray, y: np.ndarray) -> Tuple[float,float]:
        """Return point on trajectory nearest to target"""
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2
        idx = np.argmin(d2)
        return x[idx], y[idx]

    def max_distance_to_target(self, x: np.ndarray, y: np.ndarray) -> float:
        return float(np.max(np.sqrt((x - self.target_x)**2 + (y - self.target_y)**2)))

    def loss(self) -> torch.Tensor:
        v, θ = self.velocity, self.angle

        # Full trajectory for max distance
        n_points = 501
        t_flight = 2*v*torch.sin(θ)/self.g
        t = torch.linspace(0, t_flight, n_points)
        x = v*torch.cos(θ)*t
        y = v*torch.sin(θ)*t - 0.5*self.g*t**2

        # Distance to target
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2
        min_dist = torch.min(torch.sqrt(d2))
        max_dist = torch.max(torch.sqrt(d2))
        self.current_max_dist = max_dist.item()
        self.current_min_dist = min_dist.item()

        # Target loss = squared distance at closest point
        target_loss = 100*min_dist**2

        # Uniform penalty for angles between 40–50 degrees
        angle_deg = θ*180/np.pi
        penalty = 0.0
        if 44.5 <= angle_deg <= 45.5:
            penalty = 10  # uniform penalty

        # Multi-objective: target + flight time + angle penalty
        total_loss = self.w_target*target_loss - self.w_time*(2*v*torch.sin(θ)/self.g) + penalty
        return total_loss

    def optimise_and_gif(
        self,
        steps: int = 5000,
        lr: float = 0.025,
        gif_name: str = "optimisation_multi_final.gif",
        save_every: int = 25,
        early_stop_tol: float = 1e-9
    ):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        frames = []
        sampled_trajectories = []
        xlim = (0, 1.5*self.target_x)
        ylim = (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Clamp values
            self.velocity.data.clamp_(0.1, 100.0)
            self.angle.data.clamp_(-np.pi/2 + 0.01, np.pi/2 - 0.01)

            v = self.velocity.item()
            θ = self.angle.item()
            v_x = v * np.cos(θ)
            v_y = v * np.sin(θ)
            x, y, apex = self.trajectory()

            nearest_point = self.nearest_point_to_target(x, y)

            # Store history
            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_angle.append(θ*180/np.pi)
            self.history_apex.append(apex)
            self.history_nearest_point.append(nearest_point)
            self.history_min_dist.append(self.current_min_dist)
            self.history_max_dist.append(self.current_max_dist)

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, "
                      f"v_x={v_x:.2f}, v_y={v_y:.2f}, min_dist={self.history_min_dist[-1]:.2f}")
                sampled_trajectories.append((x, y, apex))

                # Plot snapshot for GIF
                fig, ax = plt.subplots()
                n_traj = len(sampled_trajectories)
                cmap = plt.get_cmap("Blues")
                for i, (xi, yi, ai) in enumerate(sampled_trajectories[:-1]):
                    alpha = 0.3 + 0.7*(i/(n_traj-1)) if n_traj > 1 else 0.5
                    ax.plot(xi, yi, color=cmap(alpha), linewidth=1)
                    ax.scatter(*ai, color="grey", s=20, alpha=0.5)
                # Latest trajectory
                xi, yi, ai = sampled_trajectories[-1]
                ax.plot(xi, yi, color="black", linewidth=2, label=f"Step {step}")
                ax.scatter(*ai, color="orange", s=40, label="Apex")
                ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
                info_text = f"v={v:.2f} m/s\nangle={θ*180/np.pi:.2f}°\nv_x={v_x:.2f}\nv_y={v_y:.2f}\nmin_dist={self.history_min_dist[-1]:.2f}"
                ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                        fontsize=10, verticalalignment='top',
                        bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
                ax.set_xlim(xlim)
                ax.set_ylim(ylim)
                ax.set_xlabel("x (m)")
                ax.set_ylabel("y (m)")
                ax.set_title(f"Trajectories up to step {step}")
                fig.tight_layout()
                fig.canvas.draw()
                frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
                frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                frames.append(frame)
                plt.close(fig)

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

        # Animate final trajectory
        x_final, y_final, _ = self.trajectory()
        n_points = len(x_final)
        step_size = max(1, n_points//100)
        for i in range(0, n_points, step_size):
            fig, ax = plt.subplots()
            for (xi, yi, _) in sampled_trajectories:
                ax.plot(xi, yi, color='blue', alpha=0.3, linewidth=1)
            ax.plot(x_final[:i+1], y_final[:i+1], color='green', linewidth=4)
            ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100)
            info_text = f"v={self.velocity.item():.2f} m/s\nangle={self.angle.item()*180/np.pi:.2f}°\n" \
                        f"v_x={self.velocity.item()*np.cos(self.angle.item()):.2f}\n" \
                        f"v_y={self.velocity.item()*np.sin(self.angle.item()):.2f}"
            ax.text(0.02, 0.95, info_text, transform=ax.transAxes,
                    fontsize=10, verticalalignment='top',
                    bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
            ax.set_xlim(xlim)
            ax.set_ylim(ylim)
            fig.tight_layout()
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=5)
        print(f"Saved {gif_name}")

        # --- Terminal plots ---
        # 1. Speed & Angle vs iteration
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(self.history_steps, self.history_v, 'b-', label='Speed (m/s)')
        ax2.plot(self.history_steps, self.history_angle, 'r-', label='Angle (deg)')
        ax1.set_xlabel('Iteration Step')
        ax1.set_ylabel('Speed (m/s)', color='b')
        ax2.set_ylabel('Angle (deg)', color='r')
        fig.tight_layout()
        plt.show()

        # 2. Nearest point to target
        fig, ax = plt.subplots()
        nearest_points_x = [p[0] for p in self.history_nearest_point]
        nearest_points_y = [p[1] for p in self.history_nearest_point]
        ax.plot(nearest_points_x, nearest_points_y, 'o-', color='purple', label='Nearest Point')
        ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100, label='Target')
        ax.set_xlabel('x (m)')
        ax.set_ylabel('y (m)')
        ax.legend()
        plt.tight_layout()
        plt.show()

        # 3. Maximum distance vs iteration
        fig, ax = plt.subplots()
        ax.plot(self.history_steps, self.history_max_dist, 'm-', label='Max Distance to Target')
        ax.set_xlabel('Iteration Step')
        ax.set_ylabel('Distance (m)')
        ax.legend()
        plt.tight_layout()
        plt.show()


# --- Example usage ---
if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(0,100)
    )
    opt.optimise_and_gif()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple

class ProjectileTargetOptimiserMulti:
    def __init__(self, target_x: float, target_y: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.target_x = target_x
        self.target_y = target_y
        print(f"Target: ({self.target_x}, {self.target_y}) | Device: {self.device}")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        # Optimisable parameters on device
        self.velocity = torch.tensor(torch.rand(1).item()*50, requires_grad=True, device=self.device)
        self.angle = torch.tensor(torch.rand(1).item()*np.pi - (np.pi/2) + 0.01, requires_grad=True, device=self.device)

        # History tracking
        self.history_steps, self.history_v, self.history_angle = [], [], []
        self.history_nearest_point, self.history_min_dist, self.history_max_dist = [], [], []

    def trajectory_gpu(self, n_points: int = 501) -> Tuple[torch.Tensor, torch.Tensor]:
        v, θ = self.velocity, self.angle
        t_flight = 2*v*torch.sin(θ)/self.g
        t = torch.linspace(0, t_flight, n_points, device=self.device)
        x = v*torch.cos(θ)*t
        y = v*torch.sin(θ)*t - 0.5*self.g*t**2
        return x, y

    def nearest_point_to_target_gpu(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2
        idx = torch.argmin(d2)
        return x[idx], y[idx]

    def max_distance_to_target_gpu(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        return torch.max(torch.sqrt((x - self.target_x)**2 + (y - self.target_y)**2))

    def loss(self) -> torch.Tensor:
        x, y = self.trajectory_gpu()
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2
        min_dist = torch.min(torch.sqrt(d2))
        max_dist = torch.max(torch.sqrt(d2))
        self.current_min_dist = min_dist.item()
        self.current_max_dist = max_dist.item()

        target_loss = 100*min_dist**2

        angle_deg = self.angle*180/np.pi
        penalty = 10.0 if 44.5 <= angle_deg <= 45.5 else 0.0

        total_loss = self.w_target*target_loss - self.w_time*(2*self.velocity*torch.sin(self.angle)/self.g) + penalty
        return total_loss

    def optimise_and_gif(self, steps=5000, lr=0.025, gif_name="optimisation_multi_final.gif",
                         save_every=25, early_stop_tol=1e-9):
        opt = optim.Adam([self.velocity, self.angle], lr=lr)
        sampled_trajectories = []
        xlim, ylim = (0, 1.5*self.target_x), (0, 1.5*self.target_y)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            self.velocity.data.clamp_(0.1, 100.0)
            self.angle.data.clamp_(-np.pi/2 + 0.01, np.pi/2 - 0.01)

            v, θ = self.velocity.item(), self.angle.item()
            x, y = self.trajectory_gpu()
            x_cpu, y_cpu = x.detach().cpu().numpy(), y.detach().cpu().numpy()  # <--- detach here
            nearest_point = self.nearest_point_to_target_gpu(x, y)
            nearest_point_cpu = (nearest_point[0].item(), nearest_point[1].item())


            # Store history
            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_angle.append(θ*180/np.pi)
            self.history_nearest_point.append(nearest_point_cpu)
            self.history_min_dist.append(self.current_min_dist)
            self.history_max_dist.append(self.current_max_dist)

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, angle={θ*180/np.pi:.2f} deg, min_dist={self.history_min_dist[-1]:.2f}")
                sampled_trajectories.append((x_cpu, y_cpu, nearest_point_cpu))

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

        # --- Fast GIF generation ---
        frames = []
        for xi, yi, ai in sampled_trajectories:
            fig, ax = plt.subplots()
            ax.plot(xi, yi, color='black', linewidth=2)
            ax.scatter(*ai, color="orange", s=40, label="Nearest Point")
            ax.scatter(self.target_x, self.target_y, c="red", marker="x", s=100, label="Target")
            ax.set_xlim(xlim)
            ax.set_ylim(ylim)
            ax.set_xlabel("x (m)")
            ax.set_ylabel("y (m)")
            fig.tight_layout()
            fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=10, loop=0)
        print(f"Saved looping GIF: {gif_name}")

        # --- Terminal plots ---
        # Speed & Angle
        fig, ax1 = plt.subplots()
        ax2 = ax1.twinx()
        ax1.plot(self.history_steps, self.history_v, 'b-', label='Speed')
        ax2.plot(self.history_steps, self.history_angle, 'r-', label='Angle')
        ax1.set_xlabel("Step"); ax1.set_ylabel("Speed (m/s)", color='b')
        ax2.set_ylabel("Angle (deg)", color='r')
        plt.tight_layout(); plt.show()

        # Nearest point trajectory
        fig, ax = plt.subplots()
        nx = [p[0] for p in self.history_nearest_point]
        ny = [p[1] for p in self.history_nearest_point]
        ax.plot(nx, ny, 'o-', color='purple', label='Nearest Point')
        ax.scatter(self.target_x, self.target_y, c='red', marker='x', s=100, label='Target')
        ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)")
        ax.legend(); plt.tight_layout(); plt.show()

        # Max distance
        fig, ax = plt.subplots()
        ax.plot(self.history_steps, self.history_max_dist, 'm-', label='Max Dist')
        ax.set_xlabel("Step"); ax.set_ylabel("Distance (m)"); ax.legend()
        plt.tight_layout(); plt.show()


if __name__ == "__main__":
    opt = ProjectileTargetOptimiserMulti(
        target_x=np.random.randint(0,100),
        target_y=np.random.randint(0,100)
    )
    opt.optimise_and_gif()

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple

from mpl_toolkits.mplot3d import Axes3D  # Needed for 3D plotting

class ProjectileTargetOptimiser3D:
    def __init__(self, target_x: float, target_y: float, target_z: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.target_x, self.target_y, self.target_z = target_x, target_y, target_z
        print(f"Target: ({target_x}, {target_y}, {target_z}) | Device: {self.device}")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        # Optimisable parameters
        self.velocity = torch.tensor(torch.rand(1).item()*50, requires_grad=True, device=self.device)
        self.theta = torch.tensor(torch.rand(1).item()*(np.pi/2-0.01)+0.01, requires_grad=True, device=self.device)  # elevation
        self.phi = torch.tensor(torch.rand(1).item()*2*np.pi, requires_grad=True, device=self.device)  # azimuth

        # History
        self.history_steps, self.history_v, self.history_theta, self.history_phi = [], [], [], []
        self.history_nearest_point, self.history_min_dist, self.history_max_dist = [], [], []

    def trajectory_gpu(self, n_points: int = 501) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        v, θ, φ = self.velocity, self.theta, self.phi
        t_flight = 2*v*torch.sin(θ)/self.g
        t = torch.linspace(0, t_flight, n_points, device=self.device)
        x = v*torch.cos(θ)*torch.cos(φ)*t
        y = v*torch.sin(θ)*t - 0.5*self.g*t**2
        z = v*torch.cos(θ)*torch.sin(φ)*t
        return x, y, z

    def nearest_point_to_target_gpu(self, x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2 + (z - self.target_z)**2
        idx = torch.argmin(d2)
        return x[idx], y[idx], z[idx]

    def max_distance_to_target_gpu(self, x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        return torch.max(torch.sqrt((x - self.target_x)**2 + (y - self.target_y)**2 + (z - self.target_z)**2))

    def loss(self) -> torch.Tensor:
        x, y, z = self.trajectory_gpu()
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2 + (z - self.target_z)**2
        min_dist = torch.min(torch.sqrt(d2))
        max_dist = torch.max(torch.sqrt(d2))
        self.current_min_dist = min_dist.item()
        self.current_max_dist = max_dist.item()

        target_loss = 100*min_dist**2
        angle_deg = self.theta*180/np.pi
        penalty = 10.0 if 44.5 <= angle_deg <= 45.5 else 0.0

        total_loss = self.w_target*target_loss - self.w_time*(2*self.velocity*torch.sin(self.theta)/self.g) + penalty
        return total_loss

    def optimise_and_gif(self, steps=5000, lr=0.025, gif_name="optimisation_3d.gif", save_every=25,
                         early_stop_tol=1e-9):
        opt = optim.Adam([self.velocity, self.theta, self.phi], lr=lr)
        sampled_trajectories = []
        xlim, ylim, zlim = (0, 1.5*self.target_x), (0, 1.5*self.target_y), (1.5*self.target_z, -1.5*self.target_z)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            self.velocity.data.clamp_(0.1, 1000.0)
            self.theta.data.clamp_(-np.pi + 0.01, np.pi - 0.01)
            self.phi.data.clamp_(0, 2*np.pi)

            v, θ, φ = self.velocity.item(), self.theta.item(), self.phi.item()
            x, y, z = self.trajectory_gpu()
            x_cpu, y_cpu, z_cpu = x.detach().cpu().numpy(), y.detach().cpu().numpy(), z.detach().cpu().numpy()
            nearest_point = self.nearest_point_to_target_gpu(x, y, z)
            nearest_point_cpu = tuple(p.item() for p in nearest_point)

            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_theta.append(θ*180/np.pi)
            self.history_phi.append(φ*180/np.pi)
            self.history_nearest_point.append(nearest_point_cpu)
            self.history_min_dist.append(self.current_min_dist)
            self.history_max_dist.append(self.current_max_dist)

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, θ={θ*180/np.pi:.2f}°, φ={φ*180/np.pi:.2f}°, min_dist={self.current_min_dist:.2f}")
                sampled_trajectories.append((x_cpu, y_cpu, z_cpu, nearest_point_cpu))

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

        # --- 3D GIF ---
        frames = []
        for xi, yi, zi, ai in sampled_trajectories:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xi, yi, zi, color='black', linewidth=2)
            ax.scatter(*ai, color='orange', s=40, label="Nearest Point")
            ax.scatter(self.target_x, self.target_y, self.target_z, c='red', marker='x', s=100, label="Target")
            ax.set_xlim(xlim); ax.set_ylim(ylim); ax.set_zlim(zlim)
            ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)"); ax.set_zlabel("z (m)")
            fig.tight_layout(); fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=10, loop=0)
        print(f"Saved looping 3D GIF: {gif_name}")

if __name__ == "__main__":
    opt = ProjectileTargetOptimiser3D(
        target_x=np.random.randint(0, 100),
        target_y=np.random.randint(0, 100),
        target_z=np.random.randint(0, 100)
    )
    opt.optimise_and_gif()

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple
from mpl_toolkits.mplot3d import Axes3D

class ProjectileTargetOptimiser3D:
    def __init__(self, target_x: float, target_y: float, target_z: float, gravity: float = 9.81,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.target_x, self.target_y, self.target_z = target_x, target_y, target_z
        print(f"Target: ({target_x}, {target_y}, {target_z}) | Device: {self.device}")
        self.g = gravity
        self.w_target = w_target
        self.w_time = w_time

        # Optimisable parameters
        self.velocity = torch.tensor(torch.rand(1).item()*50, requires_grad=True, device=self.device)
        self.theta = torch.tensor(torch.rand(1).item()*(np.pi/2-0.01)+0.01, requires_grad=True, device=self.device)
        self.phi = torch.tensor(torch.rand(1).item()*2*np.pi, requires_grad=True, device=self.device)

        # History
        self.history_steps, self.history_v, self.history_theta, self.history_phi = [], [], [], []
        self.history_nearest_point, self.history_min_dist, self.history_max_dist = [], [], []

    def trajectory_gpu(self, n_points: int = 501) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        v, θ, φ = self.velocity, self.theta, self.phi
        t_flight = 2*v*torch.sin(θ)/self.g
        t = torch.linspace(0, t_flight, n_points, device=self.device)
        x = v*torch.cos(θ)*torch.cos(φ)*t
        y = v*torch.sin(θ)*t - 0.5*self.g*t**2
        z = v*torch.cos(θ)*torch.sin(φ)*t
        return x, y, z

    def nearest_point_to_target_gpu(self, x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2 + (z - self.target_z)**2
        idx = torch.argmin(d2)
        return x[idx], y[idx], z[idx]

    def max_distance_to_target_gpu(self, x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        return torch.max(torch.sqrt((x - self.target_x)**2 + (y - self.target_y)**2 + (z - self.target_z)**2))

    def loss(self) -> torch.Tensor:
        x, y, z = self.trajectory_gpu()
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2 + (z - self.target_z)**2
        min_dist = torch.min(torch.sqrt(d2))
        max_dist = torch.max(torch.sqrt(d2))
        self.current_min_dist = min_dist.item()
        self.current_max_dist = max_dist.item()

        target_loss = 100*min_dist**2
        angle_deg = self.theta*180/np.pi
        penalty = 10.0 if 44.5 <= angle_deg <= 45.5 else 0.0

        total_loss = self.w_target*target_loss - self.w_time*(2*self.velocity*torch.sin(self.theta)/self.g) + penalty
        return total_loss

    def optimise_and_gif(self, steps=5000, lr=0.025, gif_name="optimisation_3d.gif", save_every=25,
                         early_stop_tol=1e-9):
        opt = optim.Adam([self.velocity, self.theta, self.phi], lr=lr)
        sampled_trajectories = []

        # Determine central limits based on largest absolute target coordinate
        limit = max(abs(self.target_x), abs(self.target_y), abs(self.target_z)) * 1.5
        xlim = (-limit, limit)
        ylim = (-limit, limit)
        zlim = (-limit, limit)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Looser clamps to allow negative trajectories
            self.velocity.data.clamp_(-100.0, 100.0)
            self.theta.data.clamp_(-np.pi/2 + 0.01, np.pi/2 - 0.01)
            self.phi.data.clamp_(-2*np.pi, 2*np.pi)

            v, θ, φ = self.velocity.item(), self.theta.item(), self.phi.item()
            x, y, z = self.trajectory_gpu()
            x_cpu, y_cpu, z_cpu = x.detach().cpu().numpy(), y.detach().cpu().numpy(), z.detach().cpu().numpy()
            nearest_point = self.nearest_point_to_target_gpu(x, y, z)
            nearest_point_cpu = tuple(p.item() for p in nearest_point)

            # Store history
            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_theta.append(θ*180/np.pi)
            self.history_phi.append(φ*180/np.pi)
            self.history_nearest_point.append(nearest_point_cpu)
            self.history_min_dist.append(self.current_min_dist)
            self.history_max_dist.append(self.current_max_dist)

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, θ={θ*180/np.pi:.2f}°, φ={φ*180/np.pi:.2f}°, min_dist={self.current_min_dist:.2f}")
                sampled_trajectories.append((x_cpu, y_cpu, z_cpu, nearest_point_cpu, v, θ, φ, self.current_min_dist))

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

        # --- Optimisation 3D GIF ---
        frames = []
        for xi, yi, zi, ai, v, θ, φ, min_dist in sampled_trajectories:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xi, yi, zi, color='black', linewidth=2)
            ax.scatter(*ai, color="orange", s=40, label="Nearest Point")
            ax.scatter(self.target_x, self.target_y, self.target_z, c='red', marker="x", s=100, label="Target")
            info_text = f"v={v:.2f} m/s\nθ={θ*180/np.pi:.2f}°\nφ={φ*180/np.pi:.2f}°\nmin_dist={min_dist:.2f}"
            ax.text2D(0.02, 0.95, info_text, transform=ax.transAxes,
                      fontsize=10, verticalalignment='top',
                      bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
            ax.set_xlim(xlim); ax.set_ylim(ylim); ax.set_zlim(zlim)
            ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)"); ax.set_zlabel("z (m)")
            fig.tight_layout(); fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=10, loop=0)
        print(f"Saved looping 3D GIF: {gif_name}")

        # --- 3D Terminal plot for nearest points evolution ---
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        nx = [p[0] for p in self.history_nearest_point]
        ny = [p[1] for p in self.history_nearest_point]
        nz = [p[2] for p in self.history_nearest_point]
        ax.plot(nx, ny, nz, 'o-', color='purple', label='Nearest Points')
        ax.scatter(self.target_x, self.target_y, self.target_z, c='red', marker='x', s=100, label='Target')
        ax.set_xlim(xlim); ax.set_ylim(ylim); ax.set_zlim(zlim)
        ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)"); ax.set_zlabel("z (m)")
        ax.legend(); plt.tight_layout(); plt.show()

        # --- Max distance vs iteration ---
        fig, ax = plt.subplots()
        ax.plot(self.history_steps, self.history_max_dist, 'm-', label='Max Dist to Target')
        ax.set_xlabel("Step"); ax.set_ylabel("Distance (m)"); ax.legend()
        plt.tight_layout(); plt.show()


if __name__ == "__main__":
    opt = ProjectileTargetOptimiser3D(
        target_x=np.random.randint(-50,50),
        target_y=np.random.randint(-50,50),
        target_z=np.random.randint(-50,50)
    )
    opt.optimise_and_gif()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple
from mpl_toolkits.mplot3d import Axes3D

class ProjectileTargetOptimiser3DOriginForce:
    def __init__(self, target_x: float, target_y: float, target_z: float,
                 force_strength: float = 9.81, force_exponent: float = 2.0,
                 w_target: float = 1.0, w_time: float = 0.05):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.target_x, self.target_y, self.target_z = target_x, target_y, target_z
        print(f"Target: ({target_x}, {target_y}, {target_z}) | Device: {self.device}")

        self.force_strength = force_strength
        self.force_exponent = force_exponent
        self.w_target = w_target
        self.w_time = w_time

        # Optimisable parameters
        self.velocity = torch.tensor(torch.rand(1).item()*50, requires_grad=True, device=self.device)
        self.theta = torch.tensor(torch.rand(1).item()*(np.pi/2-0.01)+0.01, requires_grad=True, device=self.device)
        self.phi = torch.tensor(torch.rand(1).item()*2*np.pi, requires_grad=True, device=self.device)

        # History
        self.history_steps, self.history_v, self.history_theta, self.history_phi = [], [], [], []
        self.history_nearest_point, self.history_min_dist, self.history_max_dist = [], [], []

    def trajectory_gpu(self, n_points: int = 501, dt: float = 0.01) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Compute trajectory with origin-seeking force; functional to avoid in-place ops"""
        v, θ, φ = self.velocity, self.theta, self.phi
        vx = v*torch.cos(θ)*torch.cos(φ)
        vy = v*torch.sin(θ)
        vz = v*torch.cos(θ)*torch.sin(φ)

        x_list = [torch.tensor(0., device=self.device)]
        y_list = [torch.tensor(0., device=self.device)]
        z_list = [torch.tensor(0., device=self.device)]
        vx_list = [vx]
        vy_list = [vy]
        vz_list = [vz]

        for _ in range(1, n_points):
            r = torch.sqrt(x_list[-1]**2 + y_list[-1]**2 + z_list[-1]**2) + 1e-6
            ax = -self.force_strength * x_list[-1] / r**self.force_exponent
            ay = -self.force_strength * y_list[-1] / r**self.force_exponent
            az = -self.force_strength * z_list[-1] / r**self.force_exponent

            vx_new = vx_list[-1] + ax*dt
            vy_new = vy_list[-1] + ay*dt
            vz_new = vz_list[-1] + az*dt

            x_new = x_list[-1] + vx_new*dt
            y_new = y_list[-1] + vy_new*dt
            z_new = z_list[-1] + vz_new*dt

            vx_list.append(vx_new)
            vy_list.append(vy_new)
            vz_list.append(vz_new)
            x_list.append(x_new)
            y_list.append(y_new)
            z_list.append(z_new)

        x = torch.stack(x_list)
        y = torch.stack(y_list)
        z = torch.stack(z_list)
        return x, y, z

    def nearest_point_to_target_gpu(self, x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2 + (z - self.target_z)**2
        idx = torch.argmin(d2)
        return x[idx], y[idx], z[idx]

    def max_distance_to_target_gpu(self, x: torch.Tensor, y: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        return torch.max(torch.sqrt((x - self.target_x)**2 + (y - self.target_y)**2 + (z - self.target_z)**2))

    def loss(self) -> torch.Tensor:
        x, y, z = self.trajectory_gpu()
        d2 = (x - self.target_x)**2 + (y - self.target_y)**2 + (z - self.target_z)**2
        min_dist = torch.min(torch.sqrt(d2))
        max_dist = torch.max(torch.sqrt(d2))
        self.current_min_dist = min_dist.item()
        self.current_max_dist = max_dist.item()

        target_loss = 100*min_dist**2
        angle_deg = self.theta*180/np.pi
        penalty = 10.0 if 44.5 <= angle_deg <= 45.5 else 0.0

        total_loss = self.w_target*target_loss - self.w_time*(self.velocity) + penalty
        return total_loss

    def optimise_and_gif(self, steps=3000, lr=0.05, gif_name="optimisation_3d_origin_force.gif",
                         save_every=20, early_stop_tol=1e-6):
        opt = optim.Adam([self.velocity, self.theta, self.phi], lr=lr)
        sampled_trajectories = []

        limit = max(abs(self.target_x), abs(self.target_y), abs(self.target_z)) * 1.5
        xlim = (-limit, limit)
        ylim = (-limit, limit)
        zlim = (-limit, limit)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Looser clamps
            self.velocity.data.clamp_(-100.0, 100.0)
            self.theta.data.clamp_(-np.pi/2 + 0.01, np.pi/2 - 0.01)
            self.phi.data.clamp_(-2*np.pi, 2*np.pi)

            v, θ, φ = self.velocity.item(), self.theta.item(), self.phi.item()
            x, y, z = self.trajectory_gpu()
            x_cpu, y_cpu, z_cpu = x.detach().cpu().numpy(), y.detach().cpu().numpy(), z.detach().cpu().numpy()
            nearest_point = self.nearest_point_to_target_gpu(x, y, z)
            nearest_point_cpu = tuple(p.item() for p in nearest_point)

            # Store history
            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_theta.append(θ*180/np.pi)
            self.history_phi.append(φ*180/np.pi)
            self.history_nearest_point.append(nearest_point_cpu)
            self.history_min_dist.append(self.current_min_dist)
            self.history_max_dist.append(self.current_max_dist)

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, θ={θ*180/np.pi:.2f}°, φ={φ*180/np.pi:.2f}°, min_dist={self.current_min_dist:.2f}")
                sampled_trajectories.append((x_cpu, y_cpu, z_cpu, nearest_point_cpu, v, θ, φ, self.current_min_dist))

            if L.item() < early_stop_tol:
                print(f"Converged at step {step}")
                break

        # --- Optimisation GIF ---
        frames = []
        for xi, yi, zi, ai, v, θ, φ, min_dist in sampled_trajectories:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xi, yi, zi, color='black', linewidth=2)
            ax.scatter(*ai, color="orange", s=40, label="Nearest Point")
            ax.scatter(self.target_x, self.target_y, self.target_z, c='red', marker="x", s=100, label="Target")
            info_text = f"v={v:.2f} m/s\nθ={θ*180/np.pi:.2f}°\nφ={φ*180/np.pi:.2f}°\nmin_dist={min_dist:.2f}"
            ax.text2D(0.02, 0.95, info_text, transform=ax.transAxes,
                      fontsize=10, verticalalignment='top',
                      bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
            ax.set_xlim(xlim); ax.set_ylim(ylim); ax.set_zlim(zlim)
            ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)"); ax.set_zlabel("z (m)")
            fig.tight_layout(); fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=10, loop=0)
        print(f"Saved looping 3D GIF: {gif_name}")

        # --- Terminal plot for nearest points evolution ---
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        nx = [p[0] for p in self.history_nearest_point]
        ny = [p[1] for p in self.history_nearest_point]
        nz = [p[2] for p in self.history_nearest_point]
        ax.plot(nx, ny, nz, 'o-', color='purple', label='Nearest Points')
        ax.scatter(self.target_x, self.target_y, self.target_z, c='red', marker='x', s=100, label='Target')
        ax.set_xlim(xlim); ax.set_ylim(ylim); ax.set_zlim(zlim)
        ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)"); ax.set_zlabel("z (m)")
        ax.legend(); plt.tight_layout(); plt.show()

        # --- Max distance vs iteration ---
        fig, ax = plt.subplots()
        ax.plot(self.history_steps, self.history_max_dist, 'm-', label='Max Dist to Target')
        ax.set_xlabel("Step"); ax.set_ylabel("Distance (m)"); ax.legend()
        plt.tight_layout(); plt.show()


if __name__ == "__main__":
    opt = ProjectileTargetOptimiser3DOriginForce(
        target_x=np.random.randint(-50,50),
        target_y=np.random.randint(-50,50),
        target_z=np.random.randint(-50,50),
        force_strength=9.81,
        force_exponent=2.0
    )
    opt.optimise_and_gif()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple

class ProjectileTargetOptimiser3DMultiAttractVectors:
    def __init__(self, target_x: float, target_y: float, target_z: float,
                 w_target: float = 1.0, w_time: float = 0.05,
                 n_forces_range: Tuple[int,int]=(1,5), G_range: Tuple[float,float]=(1,20),
                 grid_range: Tuple[int,int]=(-50,50)):
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.target = torch.tensor([target_x, target_y, target_z], dtype=torch.float32, device=self.device)
        print(f"Target: {self.target.cpu().numpy()} | Device: {self.device}")

        self.w_target = w_target
        self.w_time = w_time

        # Random attractive points
        self.n_forces = np.random.randint(n_forces_range[0], n_forces_range[1]+1)
        self.attract_points = torch.tensor(
            np.random.uniform(grid_range[0], grid_range[1], size=(self.n_forces,3)),
            dtype=torch.float32, device=self.device
        )
        self.Gs = torch.tensor(
            np.random.uniform(G_range[0], G_range[1], size=(self.n_forces,)),
            dtype=torch.float32, device=self.device
        )
        print(f"{self.n_forces} attractors, strengths: {self.Gs.cpu().numpy()}")
        print(f"Positions:\n{self.attract_points.cpu().numpy()}")

        # Optimisable parameters
        self.velocity = torch.tensor(torch.rand(1).item()*50, requires_grad=True, device=self.device)
        self.theta = torch.tensor(torch.rand(1).item()*(np.pi/2-0.01)+0.01, requires_grad=True, device=self.device)
        self.phi = torch.tensor(torch.rand(1).item()*2*np.pi, requires_grad=True, device=self.device)

        # History
        self.history_steps, self.history_v, self.history_theta, self.history_phi = [], [], [], []
        self.history_nearest_point, self.history_min_dist, self.history_max_dist = [], [], []

    # --- Vectorised RK4 integration ---
    def trajectory_rk4_gpu(self, dt=0.01, n_steps=1000):
        pos = torch.zeros(3, device=self.device)
        v0 = self.velocity
        θ, φ = self.theta, self.phi
        vel = torch.tensor([v0*torch.cos(θ)*torch.cos(φ),
                            v0*torch.sin(θ),
                            v0*torch.cos(θ)*torch.sin(φ)], device=self.device)

        traj = torch.zeros((n_steps,3), device=self.device)

        for i in range(n_steps):
            traj[i] = pos

            def acceleration(p, vel):
                r_vecs = self.attract_points - p.unsqueeze(0)  # (n_forces,3)
                r_mag = torch.norm(r_vecs, dim=1, keepdim=True)
                return torch.sum(self.Gs.unsqueeze(1) * r_vecs / (r_mag**3 + 1e-6), dim=0)

            k1_v = acceleration(pos, vel) * dt
            k1_x = vel * dt

            k2_v = acceleration(pos + 0.5*k1_x, vel + 0.5*k1_v) * dt
            k2_x = (vel + 0.5*k1_v) * dt

            k3_v = acceleration(pos + 0.5*k2_x, vel + 0.5*k2_v) * dt
            k3_x = (vel + 0.5*k2_v) * dt

            k4_v = acceleration(pos + k3_x, vel + k3_v) * dt
            k4_x = (vel + k3_v) * dt

            pos = pos + (k1_x + 2*k2_x + 2*k3_x + k4_x)/6
            vel = vel + (k1_v + 2*k2_v + 2*k3_v + k4_v)/6

        return traj[:,0], traj[:,1], traj[:,2]

    def nearest_point_to_target_gpu(self, x, y, z):
        d2 = (x - self.target[0])**2 + (y - self.target[1])**2 + (z - self.target[2])**2
        idx = torch.argmin(d2)
        return x[idx], y[idx], z[idx]

    def loss(self):
        x, y, z = self.trajectory_rk4_gpu()
        d2 = (x - self.target[0])**2 + (y - self.target[1])**2 + (z - self.target[2])**2
        min_dist = torch.min(torch.sqrt(d2))
        max_dist = torch.max(torch.sqrt(d2))
        self.current_min_dist = min_dist.item()
        self.current_max_dist = max_dist.item()
        target_loss = 100*min_dist**2
        penalty = 10.0 if 44.5 <= self.theta*180/np.pi <= 45.5 else 0.0
        return self.w_target*target_loss - self.w_time*self.velocity + penalty

    def optimise_and_gif(self, steps=100, lr=0.5, gif_name="optimisation_3d_multi_vector.gif", save_every=5):
        opt = optim.Adam([self.velocity, self.theta, self.phi], lr=lr)
        sampled_trajectories = []

        limit = max(abs(self.target[0]), abs(self.target[1]), abs(self.target[2]))*1.5
        limit = float(max(abs(self.target[0].item()), abs(self.target[1].item()), abs(self.target[2].item())) * 1.5)
        xlim = (-limit, limit)
        ylim = (-limit, limit)
        zlim = (-limit, limit)


        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Looser clamps
            self.velocity.data.clamp_(-100,100)
            self.theta.data.clamp_(-np.pi/2+0.01, np.pi/2-0.01)
            self.phi.data.clamp_(-2*np.pi, 2*np.pi)

            v, θ, φ = self.velocity.item(), self.theta.item(), self.phi.item()
            x, y, z = self.trajectory_rk4_gpu()
            x_cpu, y_cpu, z_cpu = x.detach().cpu().numpy(), y.detach().cpu().numpy(), z.detach().cpu().numpy()
            nearest_point = self.nearest_point_to_target_gpu(x, y, z)
            nearest_point_cpu = tuple(p.item() for p in nearest_point)

            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_theta.append(θ*180/np.pi)
            self.history_phi.append(φ*180/np.pi)
            self.history_nearest_point.append(nearest_point_cpu)
            self.history_min_dist.append(self.current_min_dist)
            self.history_max_dist.append(self.current_max_dist)

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, θ={θ*180/np.pi:.2f}°, φ={φ*180/np.pi:.2f}°, min_dist={self.current_min_dist:.2f}")
                sampled_trajectories.append((x_cpu, y_cpu, z_cpu, nearest_point_cpu, v, θ, φ, self.current_min_dist))

        # --- GIF with force vectors ---
        frames = []
        for xi, yi, zi, ai, v, θ, φ, min_dist in sampled_trajectories:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xi, yi, zi, color='black', linewidth=2)
            ax.scatter(*ai, color='orange', s=40, label='Nearest Point')
            ax.scatter(*self.target.cpu().numpy(), c='red', marker='x', s=100, label='Target')
            # Force vectors
            for pos, G in zip(self.attract_points.cpu().numpy(), self.Gs.cpu().numpy()):
                ax.quiver(pos[0], pos[1], pos[2],
                          -pos[0]*0.1, -pos[1]*0.1, -pos[2]*0.1, color='blue', length=5, normalize=True)
            info_text = f"v={v:.2f} m/s\nθ={θ*180/np.pi:.2f}°\nφ={φ*180/np.pi:.2f}°\nmin_dist={min_dist:.2f}"
            ax.text2D(0.02, 0.95, info_text, transform=ax.transAxes,
                      fontsize=10, verticalalignment='top',
                      bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
            ax.set_xlim(xlim); ax.set_ylim(ylim); ax.set_zlim(zlim)
            ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)"); ax.set_zlabel("z (m)")
            fig.tight_layout(); fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=10, loop=0)
        print(f"Saved looping 3D GIF: {gif_name}")

# --- Run ---
if __name__ == "__main__":
    opt = ProjectileTargetOptimiser3DMultiAttractVectors(
        target_x=np.random.randint(-50,50),
        target_y=np.random.randint(-50,50),
        target_z=np.random.randint(-50,50)
    )
    opt.optimise_and_gif()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from typing import Tuple
from mpl_toolkits.mplot3d import Axes3D

class ProjectileTargetOptimiser3DMultiAttractVectors:
    def __init__(self, target_x: float, target_y: float, target_z: float,
                 w_target: float = 1.0, w_time: float = 0.01,
                 n_forces_range: Tuple[int,int]=(1,5), G_range: Tuple[float,float]=(1,20),
                 grid_range: Tuple[int,int]=(-50,50)):

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.target = torch.tensor([target_x, target_y, target_z], dtype=torch.float32, device=self.device)
        print(f"Target: {self.target.cpu().numpy()} | Device: {self.device}")

        self.w_target = w_target
        self.w_time = w_time

        # Random attractive points
        self.n_forces = np.random.randint(n_forces_range[0], n_forces_range[1]+1)
        self.attract_points = torch.tensor(
            np.random.uniform(grid_range[0], grid_range[1], size=(self.n_forces,3)),
            dtype=torch.float32, device=self.device
        )
        self.Gs = torch.tensor(
            np.random.uniform(G_range[0], G_range[1], size=(self.n_forces,)),
            dtype=torch.float32, device=self.device
        )
        print(f"{self.n_forces} attractors, strengths: {self.Gs.cpu().numpy()}")
        print(f"Positions:\n{self.attract_points.cpu().numpy()}")

        # Optimisable parameters
        self.velocity = torch.tensor(torch.rand(1).item()*50, requires_grad=True, device=self.device)
        self.theta = torch.tensor(torch.rand(1).item()*(np.pi/2-0.01)+0.01, requires_grad=True, device=self.device)
        self.phi = torch.tensor(torch.rand(1).item()*2*np.pi, requires_grad=True, device=self.device)

        # History
        self.history_steps, self.history_v, self.history_theta, self.history_phi = [], [], [], []
        self.history_nearest_point, self.history_min_dist, self.history_max_dist = [], [], []

    # --- Differentiable RK4 integration ---
    def trajectory_rk4_gpu(self, dt=0.01, n_steps=1000):
        pos = torch.zeros(3, device=self.device)
        v0 = self.velocity
        θ, φ = self.theta, self.phi
        vel = torch.tensor([v0*torch.cos(θ)*torch.cos(φ),
                            v0*torch.sin(θ),
                            v0*torch.cos(θ)*torch.sin(φ)], device=self.device)

        traj = torch.zeros((n_steps,3), device=self.device)

        for i in range(n_steps):
            traj[i] = pos

            def acceleration(p):
                r_vecs = self.attract_points - p.unsqueeze(0)  # (n_forces,3)
                r_mag = torch.norm(r_vecs, dim=1, keepdim=True) + 1e-6
                return torch.sum(self.Gs.unsqueeze(1) * r_vecs / (r_mag**3), dim=0)

            k1_v = acceleration(pos) * dt
            k1_x = vel * dt

            k2_v = acceleration(pos + 0.5*k1_x) * dt
            k2_x = (vel + 0.5*k1_v) * dt

            k3_v = acceleration(pos + 0.5*k2_x) * dt
            k3_x = (vel + 0.5*k2_v) * dt

            k4_v = acceleration(pos + k3_x) * dt
            k4_x = (vel + k3_v) * dt

            pos = pos + (k1_x + 2*k2_x + 2*k3_x + k4_x)/6
            vel = vel + (k1_v + 2*k2_v + 2*k3_v + k4_v)/6

        return traj[:,0], traj[:,1], traj[:,2]

    def nearest_point_to_target_gpu(self, x, y, z):
        d2 = (x - self.target[0])**2 + (y - self.target[1])**2 + (z - self.target[2])**2
        idx = torch.argmin(d2)
        return x[idx], y[idx], z[idx]

    def loss(self):
        x, y, z = self.trajectory_rk4_gpu()
        d2 = (x - self.target[0])**2 + (y - self.target[1])**2 + (z - self.target[2])**2
        min_dist = torch.min(torch.sqrt(d2))
        max_dist = torch.max(torch.sqrt(d2))
        self.current_min_dist = min_dist.item()
        self.current_max_dist = max_dist.item()

        target_loss = self.w_target * min_dist**2
        velocity_penalty = self.w_time * self.velocity**2
        return target_loss + velocity_penalty

    def optimise_and_gif(self, steps=500, lr=0.05, gif_name="optimisation_3d_multi_vector.gif", save_every=20):
        opt = optim.Adam([self.velocity, self.theta, self.phi], lr=lr)
        sampled_trajectories = []

        # Central limits for plotting
        limit = float(max(torch.abs(self.target).cpu().numpy())) * 1.5
        xlim = (-limit, limit)
        ylim = (-limit, limit)
        zlim = (-limit, limit)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # Looser clamps
            self.velocity.data.clamp_(-100,100)
            self.theta.data.clamp_(-np.pi/2+0.01, np.pi/2-0.01)
            self.phi.data.clamp_(-2*np.pi, 2*np.pi)

            v, θ, φ = self.velocity.item(), self.theta.item(), self.phi.item()
            x, y, z = self.trajectory_rk4_gpu()
            x_cpu, y_cpu, z_cpu = x.detach().cpu().numpy(), y.detach().cpu().numpy(), z.detach().cpu().numpy()
            nearest_point = self.nearest_point_to_target_gpu(x, y, z)
            nearest_point_cpu = tuple(p.item() for p in nearest_point)

            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_theta.append(θ*180/np.pi)
            self.history_phi.append(φ*180/np.pi)
            self.history_nearest_point.append(nearest_point_cpu)
            self.history_min_dist.append(self.current_min_dist)
            self.history_max_dist.append(self.current_max_dist)

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, θ={θ*180/np.pi:.2f}°, φ={φ*180/np.pi:.2f}°, min_dist={self.current_min_dist:.2f}")
                sampled_trajectories.append((x_cpu, y_cpu, z_cpu, nearest_point_cpu, v, θ, φ, self.current_min_dist))

        # --- GIF ---
        frames = []
        for xi, yi, zi, ai, v, θ, φ, min_dist in sampled_trajectories:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xi, yi, zi, color='black', linewidth=2)
            ax.scatter(*ai, color='orange', s=40, label='Nearest Point')
            ax.scatter(*self.target.cpu().numpy(), c='red', marker='x', s=100, label='Target')

            # Force vectors
            for pos, G in zip(self.attract_points.cpu().numpy(), self.Gs.cpu().numpy()):
                ax.quiver(pos[0], pos[1], pos[2],
                          -pos[0]*0.1, -pos[1]*0.1, -pos[2]*0.1, color='blue', length=5, normalize=True)

            info_text = f"v={v:.2f} m/s\nθ={θ*180/np.pi:.2f}°\nφ={φ*180/np.pi:.2f}°\nmin_dist={min_dist:.2f}"
            ax.text2D(0.02, 0.95, info_text, transform=ax.transAxes,
                      fontsize=10, verticalalignment='top',
                      bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

            ax.set_xlim(xlim); ax.set_ylim(ylim); ax.set_zlim(zlim)
            ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)"); ax.set_zlabel("z (m)")
            fig.tight_layout(); fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=10, loop=0)
        print(f"Saved looping 3D GIF: {gif_name}")


# --- Run ---
if __name__ == "__main__":
    opt = ProjectileTargetOptimiser3DMultiAttractVectors(
        target_x=np.random.randint(-50,50),
        target_y=np.random.randint(-50,50),
        target_z=np.random.randint(-50,50)
    )
    opt.optimise_and_gif()


In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple

class ProjectileTargetOptimiser3DMultiAttractVectors:
    def __init__(self, target_x: float, target_y: float, target_z: float,
                 w_target: float = 1.0, w_time: float = 0.01,
                 n_forces_range: Tuple[int,int]=(3,10), G_range: Tuple[float,float]=(250,500),
                 grid_range: Tuple[int,int]=(-100,100)):

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.target = torch.tensor([target_x, target_y, target_z], dtype=torch.float32, device=self.device)
        print(f"Target: {self.target.cpu().numpy()} | Device: {self.device}")

        self.w_target = w_target
        self.w_time = w_time

        # Random attractors
        self.n_forces = np.random.randint(n_forces_range[0], n_forces_range[1]+1)
        self.attract_points = torch.tensor(
            np.random.uniform(grid_range[0], grid_range[1], size=(self.n_forces,3)),
            dtype=torch.float32, device=self.device
        )
        self.Gs = torch.tensor(
            np.random.uniform(G_range[0], G_range[1], size=(self.n_forces,)),
            dtype=torch.float32, device=self.device
        )
        print(f"{self.n_forces} attractors, strengths: {self.Gs.cpu().numpy()}")
        print(f"Positions:\n{self.attract_points.cpu().numpy()}")

        # Optimisable parameters
        self.velocity = torch.tensor(0.0 + 5.0*torch.rand(1).item(), requires_grad=True, device=self.device)
        self.theta = torch.tensor(torch.rand(1).item()*(np.pi/2-0.01)+0.01, requires_grad=True, device=self.device)
        self.phi = torch.tensor(torch.rand(1).item()*2*np.pi, requires_grad=True, device=self.device)

        # History
        self.history_steps, self.history_v, self.history_theta, self.history_phi = [], [], [], []
        self.history_nearest_point, self.history_min_dist, self.history_max_dist = [], [], []

    def trajectory_rk4_gpu(self, dt=2, n_steps=1000):
        pos = torch.zeros(3, device=self.device)
        θ, φ = self.theta, self.phi
        v0 = self.velocity
        vel = torch.tensor([v0*torch.cos(θ)*torch.cos(φ),
                            v0*torch.sin(θ),
                            v0*torch.cos(θ)*torch.sin(φ)], device=self.device)

        traj = torch.zeros((n_steps,3), device=self.device)

        for i in range(n_steps):
            traj[i] = pos

            def acceleration(p):
                r_vecs = self.attract_points - p.unsqueeze(0)  # (n_forces,3)
                r_mag = torch.norm(r_vecs, dim=1, keepdim=True)
                return torch.sum(self.Gs.unsqueeze(1) * r_vecs / (r_mag**3 + 1e-6), dim=0)

            k1_v = acceleration(pos) * dt
            k1_x = vel * dt
            k2_v = acceleration(pos + 0.5*k1_x) * dt
            k2_x = (vel + 0.5*k1_v) * dt
            k3_v = acceleration(pos + 0.5*k2_x) * dt
            k3_x = (vel + 0.5*k2_v) * dt
            k4_v = acceleration(pos + k3_x) * dt
            k4_x = (vel + k3_v) * dt

            pos = pos + (k1_x + 2*k2_x + 2*k3_x + k4_x)/6
            vel = vel + (k1_v + 2*k2_v + 2*k3_v + k4_v)/6

        return traj[:,0], traj[:,1], traj[:,2]

    def nearest_point_to_target_gpu(self, x, y, z):
        d2 = (x - self.target[0])**2 + (y - self.target[1])**2 + (z - self.target[2])**2
        idx = torch.argmin(d2)
        return x[idx], y[idx], z[idx]

    def loss(self):
        x, y, z = self.trajectory_rk4_gpu()
        d2 = (x - self.target[0])**2 + (y - self.target[1])**2 + (z - self.target[2])**2
        # smooth soft-min loss
        w = torch.exp(-d2)
        smooth_min = torch.sum(d2 * w) / torch.sum(w)
        
        self.current_min_dist = torch.sqrt(torch.min(d2)).item()
        self.current_max_dist = torch.sqrt(torch.max(d2)).item()
        penalty = 10.0 if 44.5 <= self.theta*180/np.pi <= 45.5 else 0.0
        
        return 1000*self.w_target*smooth_min - self.w_time*self.velocity + penalty

    def optimise_and_gif(self, steps=10, lr_v=5, lr_angles=5, gif_name="optimisation_3d_multi_vector.gif", save_every=2):
        opt = optim.Adam([
            {'params':[self.velocity], 'lr': lr_v},
            {'params':[self.theta, self.phi], 'lr': lr_angles}
        ])
        sampled_trajectories = []

        limit = float(max(abs(self.target[0].item()), abs(self.target[1].item()), abs(self.target[2].item())) * 1.5)
        xlim = (-limit, limit)
        ylim = (-limit, limit)
        zlim = (-limit, limit)

        for step in range(steps):
            opt.zero_grad()
            L = self.loss()
            L.backward()
            opt.step()

            # clamp ranges
            self.velocity.data.clamp_(1,5)
            self.theta.data.clamp_(-np.pi, np.pi)
            self.phi.data.clamp_(-np.pi, np.pi)

            v, θ, φ = self.velocity.item(), self.theta.item(), self.phi.item()
            x, y, z = self.trajectory_rk4_gpu()
            x_cpu, y_cpu, z_cpu = x.detach().cpu().numpy(), y.detach().cpu().numpy(), z.detach().cpu().numpy()
            nearest_point = self.nearest_point_to_target_gpu(x, y, z)
            nearest_point_cpu = tuple(p.item() for p in nearest_point)

            self.history_steps.append(step)
            self.history_v.append(v)
            self.history_theta.append(θ*180/np.pi)
            self.history_phi.append(φ*180/np.pi)
            self.history_nearest_point.append(nearest_point_cpu)
            self.history_min_dist.append(self.current_min_dist)
            self.history_max_dist.append(self.current_max_dist)

            if step % save_every == 0:
                print(f"Step {step}: v={v:.2f}, θ={θ*180/np.pi:.2f}°, φ={φ*180/np.pi:.2f}°, min_dist={self.current_min_dist:.2f}")
                sampled_trajectories.append((x_cpu, y_cpu, z_cpu, nearest_point_cpu, v, θ, φ, self.current_min_dist))

        # --- GIF generation ---
        frames = []
        for xi, yi, zi, ai, v, θ, φ, min_dist in sampled_trajectories:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xi, yi, zi, color='black', linewidth=2)
            ax.scatter(*ai, color='orange', s=40, label='Nearest Point')
            ax.scatter(*self.target.cpu().numpy(), c='red', marker='x', s=100, label='Target')
            for pos, G in zip(self.attract_points.cpu().numpy(), self.Gs.cpu().numpy()):
                ax.quiver(pos[0], pos[1], pos[2],
                          -pos[0]*0.1, -pos[1]*0.1, -pos[2]*0.1, color='blue', length=5, normalize=True)
            info_text = f"v={v:.2f} m/s\nθ={θ*180/np.pi:.2f}°\nφ={φ*180/np.pi:.2f}°\nmin_dist={min_dist:.2f}"
            ax.text2D(0.02, 0.95, info_text, transform=ax.transAxes,
                      fontsize=10, verticalalignment='top',
                      bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
            ax.set_xlim(xlim); ax.set_ylim(ylim); ax.set_zlim(zlim)
            ax.set_xlabel("x (m)"); ax.set_ylabel("y (m)"); ax.set_zlabel("z (m)")
            fig.tight_layout(); fig.canvas.draw()
            frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            frames.append(frame)
            plt.close(fig)

        imageio.mimsave(gif_name, frames, fps=10, loop=0)
        print(f"Saved looping 3D GIF: {gif_name}")


# --- Run ---
if __name__ == "__main__":
    opt = ProjectileTargetOptimiser3DMultiAttractVectors(
        target_x=np.random.randint(0,50),
        target_y=np.random.randint(0,50),
        target_z=np.random.randint(0,50)
    )
    opt.optimise_and_gif()
